# Ноутбук с решением кейса

Итоги всех моделей: 


| Модель                    | ndcg@10 | recall@10 | map@10 | serendipity@10 | coverage |
| :------------------------ | :------ | :-------- | :----- | :------------- | :------- |
| **Item2item + BM25**      | 0.090   | 0.072     | 0.049  | 0.081          | 0.615    |
| **LightGBM Reranker**     | 0.029   | 0.023     | 0.012  | 0.031          | 0.024    |
| **EASE_2**                | 0.028   | 0.023     | 0.013  | 0.021          | 0.264    |
| **EASE_1**                | 0.021   | 0.016     | 0.009  | 0.018          | 0.398    |
| **MultiChannelRecommender** | 0.014   | 0.009     | 0.006  | 0.013          | 0.254    |
| **SVD**                   | 0.011   | 0.007     | 0.004  | 0.012          | 0.038    |



In [1]:
import pandas as pd
import polars as pl
import numpy as np
import typing as t
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
#from tqdm.notebook import tqdm
from tqdm import tqdm
from IPython.display import HTML
from dataclasses import dataclass
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize


def dataframe_to_html(df: pl.DataFrame, columns: list) -> str:
    """
    Convert a Polars DataFrame to an HTML table with specified columns.

    Args:
        df: Polars DataFrame containing the data.
        columns: List of column names to include in the HTML table.

    Returns:
        str: HTML string representing the table.
    """
    html = ["<table border='1'>"]

    html.append("<tr>")
    for col in columns:
        html.append(f"<th>{col}</th>")
    html.append("</tr>")

    for row in df.iter_rows(named=True):
        html.append("<tr>")
        for col in columns:
            value = row[col]
            if col == "image_url":
                html.append(f"<td><img src='{value}' width='100' /></td>")
            else:
                html.append(f"<td>{value}</td>")
        html.append("</tr>")

    html.append("</table>")
    return "\n".join(html)


In [2]:
data_folder = "data/"

train = pl.read_parquet(data_folder + "train.pq")
test_exploded = pl.read_parquet(data_folder + "test.pq")
test = test_exploded.group_by("user_id", maintain_order=True).agg(pl.col("item_id"))

In [3]:
books = pl.read_parquet(data_folder + "books.pq")

In [4]:
n_books = books.shape[0]
print(f"N books: {n_books}")

n_cold_books = n_books - train["item_id"].n_unique()
print(f"N 'cold' books: {n_cold_books}")

N books: 34322
N 'cold' books: 3022


In [5]:
# ACHTUNG! DO NOT TOUCH

@dataclass
class AtKMetric(ABC):
    k: int

    @property
    @abstractmethod
    def name(self) -> str:
        raise NotImplementedError

    @property
    def full_name(self) -> str:
        return f"{self.name}@{self.k}"

    @abstractmethod
    def __call__(self, *, preds_col: str = "preds", ground_truth_col: str = "ground_truth") -> pl.Expr:
        raise NotImplementedError


class NDCG(AtKMetric):
    @property
    def name(self) -> str:
        return "ndcg"

    def __call__(self, *, preds_col: str = "preds", ground_truth_col: str = "ground_truth") -> pl.Expr:
        def _dcg(scores: np.ndarray) -> float:
            # Handle empty arrays
            if len(scores) == 0:
                return 0.0
            # Use vectorized operations for better performance
            positions = np.arange(1, len(scores) + 1, dtype=np.float64)
            return np.sum((np.power(2, scores) - 1) / np.log2(positions + 1))

        def ndcg(predicted: list[t.Any], gt_items: list[t.Any]) -> float:
            # Handle empty predictions or ground truth
            if not predicted or not gt_items:
                return 0.0
                
            # Take only top-k predictions
            predicted = predicted[:self.k]
            
            # Create relevance scores (1 for relevant items, 0 otherwise)
            relevance = np.array([1 if x in gt_items else 0 for x in predicted], dtype=np.float64)
            
            rank_dcg = _dcg(relevance)
            if rank_dcg == 0.0:
                return 0.0
                
            # Ideal DCG: sort ground truth by relevance (all 1s) and take top-k
            ideal_relevance = np.ones(min(len(gt_items), self.k), dtype=np.float64)
            ideal_dcg = _dcg(ideal_relevance)
            
            if ideal_dcg == 0.0:
                return 0.0
                
            return rank_dcg / ideal_dcg

        return pl.struct([preds_col, ground_truth_col]).map_elements(
            lambda x: ndcg(x[preds_col], x[ground_truth_col]),
            return_dtype=pl.Float64
        ).alias(self.full_name)


class Recall(AtKMetric):
    @property
    def name(self) -> str:
        return "recall"

    def __call__(self, *, preds_col: str = "preds", ground_truth_col: str = "ground_truth") -> pl.Expr:
        def recall(predicted: list[t.Any], gt_items: list[t.Any]) -> float:
            # Handle empty ground truth
            if not gt_items:
                return 0.0
                
            # Take only top-k predictions
            predicted = predicted[:self.k]
            
            # Calculate intersection
            intersection = len(set(gt_items).intersection(set(predicted)))
            return intersection / len(gt_items)

        return pl.struct([preds_col, ground_truth_col]).map_elements(
            lambda x: recall(x[preds_col], x[ground_truth_col]),
            return_dtype=pl.Float64
        ).alias(self.full_name)


def coverage(df: pl.DataFrame, n_items_in_catalog: int, preds_col: str = "preds") -> float:
    if n_items_in_catalog <= 0:
        return 0.0
        
    unique_recommended = df.select(
        pl.col(preds_col).explode().n_unique()
    ).item()
    return unique_recommended / n_items_in_catalog


def evaluate_recommender(
    df: pl.DataFrame,
    model_preds_col: str,
    ground_truth_col: str = "item_id",
    n_items_in_catalog: int = n_books,
    k: int = 10,
) -> dict:
    metrics = [
        NDCG(k=k),
        Recall(k=k),
    ]
    result = {}
    
    # Create a copy to avoid modifying the original dataframe
    result_df = df.clone()
    
    for metric in metrics:
        result_df = result_df.with_columns(
            metric(preds_col=model_preds_col, ground_truth_col=ground_truth_col)
        )
        result[metric.full_name] = round(result_df.select(pl.col(metric.full_name).mean()).item(), 3)

    if n_items_in_catalog is not None:
        result["coverage"] = round(coverage(result_df, n_items_in_catalog, preds_col=model_preds_col), 3)
    
    return result

In [6]:
class MeanAveragePrecision(AtKMetric):
    @property
    def name(self) -> str:
        return "map"

    def __call__(self, *, preds_col: str = "preds", ground_truth_col: str = "ground_truth") -> pl.Expr:
        def map_k(predicted: list[t.Any], gt_items: list[t.Any]) -> float:
            if not predicted or not gt_items:
                return 0.0
            
            predicted = predicted[:self.k]
            gt_set = set(gt_items)
            
            avg_precision = 0.0
            hits = 0
            
            for i, p in enumerate(predicted):
                if p in gt_set:
                    hits += 1
                    avg_precision += hits / (i + 1)
            
            return avg_precision / min(len(gt_items), self.k)

        return pl.struct([preds_col, ground_truth_col]).map_elements(
            lambda x: map_k(x[preds_col], x[ground_truth_col]),
            return_dtype=pl.Float64
        ).alias(self.full_name)

class Serendipity(AtKMetric):
    def __init__(self, k: int, baseline_col: str = "toppopular_recs"):
        super().__init__(k=k)
        self.baseline_col = baseline_col
        
    @property
    def name(self) -> str:
        return "serendipity"
        
    def __call__(self, *, preds_col: str = "preds", ground_truth_col: str = "ground_truth") -> pl.Expr:
        def compute_serendipity(row):
            preds = row[preds_col][:self.k]
            ground_truth = row[ground_truth_col]
            baseline = row[self.baseline_col][:self.k]
            
            #не из TopPopular и релевантные 
            unexpected_correct = set(preds) - set(baseline)
            unexpected_correct &= set(ground_truth)
            
            if not ground_truth:
                return 0.0
                
            # нормализуем по числу релевантных 
            return len(unexpected_correct) / min(len(ground_truth), self.k)
            
        return pl.struct([preds_col, ground_truth_col, self.baseline_col]).map_elements(
            lambda x: compute_serendipity(x),
            return_dtype=pl.Float64
        ).alias(self.full_name)


def evaluate_recommender_modified(
    df: pl.DataFrame,
    model_preds_col: str,
    ground_truth_col: str = "item_id",
    n_items_in_catalog: int = n_books,
    k: int = 10,
    baseline_col: str = "toppopular_recs"  
) -> dict:
    metrics = [
        NDCG(k=k),
        Recall(k=k),
        MeanAveragePrecision(k=k),
    ]
    
    if baseline_col in df.columns and baseline_col != model_preds_col:
        metrics.append(Serendipity(k=k, baseline_col=baseline_col))
    
    result = {}
    
    # Create a copy to avoid modifying the original dataframe
    result_df = df.clone()
    
    for metric in metrics:
        result_df = result_df.with_columns(
            metric(preds_col=model_preds_col, ground_truth_col=ground_truth_col)
        )
        result[metric.full_name] = round(result_df.select(pl.col(metric.full_name).mean()).item(), 3)

    if n_items_in_catalog is not None:
        result["coverage"] = round(coverage(result_df, n_items_in_catalog, preds_col=model_preds_col), 3)
            
    return result

## Запустим baseline еше раз для сравнения

In [7]:
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pl.DataFrame, **kwargs) -> None:
        # реализация может быть любой, никаких ограничений
        # не забудьте про self.trained = True
        self.trained = True

    @abstractmethod
    def predict(self, df: pl.DataFrame, topn: int = 10, **kwargs) -> list[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращаться список массивов из item_id, которые есть в `books`, чтобы корректно работал подсчет метрик
        pass

In [8]:
class RandomRecommender:
    def __init__(self):
        self.trained = False
        self.item_ids = None

    def fit(self, df: pl.DataFrame, item_id_col: str = "item_id") -> None:
        # сохраняем уникальные айтемы
        self.item_ids = df[item_id_col].unique().to_numpy()
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10) -> list[np.ndarray]:
        assert self.trained
        # случайно выбираем `topn` айтемов
        preds = []
        for _ in tqdm(range(len(df))):
            preds.append(np.random.choice(self.item_ids, size=topn, replace=False))
        return preds


random_rec = RandomRecommender()
random_rec.fit(books)  # выбираем из всего доступного каталога
test = test.with_columns(
    random_recs=pl.Series(random_rec.predict(test))
)
print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="random_recs")

100%|██████████| 185828/185828 [00:59<00:00, 3125.10it/s]


shape: (5, 3)
┌─────────────────────────────────┬───────────────────────┬─────────────────────────┐
│ user_id                         ┆ item_id               ┆ random_recs             │
│ ---                             ┆ ---                   ┆ ---                     │
│ str                             ┆ list[i64]             ┆ array[i64, 10]          │
╞═════════════════════════════════╪═══════════════════════╪═════════════════════════╡
│ 00000377eea48021d3002730d56aca… ┆ [13252]               ┆ [4549, 3786, … 23]      │
│ 00009ab2ed8cbfceda5a59da409663… ┆ [2328]                ┆ [17711, 20522, … 14156] │
│ 00009e46d18f223a82b22da38586b6… ┆ [28636, 30197]        ┆ [30088, 5896, … 9325]   │
│ 0001085188e302fc6b2568de45a5f5… ┆ [2159, 2969, … 33630] ┆ [26856, 18786, … 5704]  │
│ 00014c578111090720e20f5705eba0… ┆ [45, 3513, … 33273]   ┆ [13082, 14100, … 7391]  │
└─────────────────────────────────┴───────────────────────┴─────────────────────────┘


{'ndcg@10': 0.001, 'recall@10': 0.0, 'map@10': 0.0, 'coverage': 1.0}

In [8]:
class TopPopular:
    def __init__(self):
        self.trained = False
        self.recommendations = None

    def fit(self, df: pl.DataFrame, item_id_col: str = "item_id") -> None:
        # считаем популярность айтемов
        self.recommendations = (
            df.lazy()
            .group_by(item_id_col)
            .agg(count=pl.len())
            .sort("count", descending=True)
            .collect()
            [item_id_col]
            .to_numpy()
        )
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10) -> list[np.ndarray]:
        assert self.trained
        # возвращаем для всех одно и то же
        return [self.recommendations[:topn]] * len(df)


toppop = TopPopular()
toppop.fit(train)
test = test.with_columns(
    toppopular_recs=pl.Series(toppop.predict(test))
)
print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="toppopular_recs")

shape: (5, 3)
┌─────────────────────────────────┬───────────────────────┬────────────────────────┐
│ user_id                         ┆ item_id               ┆ toppopular_recs        │
│ ---                             ┆ ---                   ┆ ---                    │
│ str                             ┆ list[i64]             ┆ array[i64, 10]         │
╞═════════════════════════════════╪═══════════════════════╪════════════════════════╡
│ 00000377eea48021d3002730d56aca… ┆ [13252]               ┆ [4058, 15514, … 13159] │
│ 00009ab2ed8cbfceda5a59da409663… ┆ [2328]                ┆ [4058, 15514, … 13159] │
│ 00009e46d18f223a82b22da38586b6… ┆ [28636, 30197]        ┆ [4058, 15514, … 13159] │
│ 0001085188e302fc6b2568de45a5f5… ┆ [2159, 2969, … 33630] ┆ [4058, 15514, … 13159] │
│ 00014c578111090720e20f5705eba0… ┆ [45, 3513, … 33273]   ┆ [4058, 15514, … 13159] │
└─────────────────────────────────┴───────────────────────┴────────────────────────┘


{'ndcg@10': 0.033, 'recall@10': 0.033, 'map@10': 0.016, 'coverage': 0.0}

In [11]:
tags_as_strings = books["tags"].map_elements(lambda x: " ".join(x), return_dtype=pl.String)

vectorizer = CountVectorizer(min_df=0.2, max_df=0.8, max_features=32)
X = vectorizer.fit_transform(tags_as_strings)
embeddings = normalize(X.toarray()).tolist()

books = books.with_columns(
    pl.Series("embedding", embeddings)
)
display(books.head())

item_id,series,tags,title,description,url,image_url,authors,embedding_new,embedding
i64,list[str],list[str],str,str,str,str,list[struct[2]],list[f64],list[f64]
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[0.550546, 0.033266, … -0.019082]","[0.0, 0.585206, … 0.0]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[0.585932, 0.176821, … -0.004595]","[0.0, 0.0, … 0.199007]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[0.5157, 0.278442, … -0.002641]","[0.0, 0.27735, … 0.0]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[0.638572, 0.326413, … -0.04838]","[0.0, 0.0, … 0.0]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[0.324899, 0.076244, … -0.013045]","[0.0, 0.2, … 0.0]"


In [12]:
class ItemkNN:
    def __init__(self):
        self.trained: bool = False
        self.train_most_liked_item: pl.DataFrame | None = None
        self.items_embeddings: np.ndarray | None = None
        self.items_item_ids: np.ndarray | None = None

    def fit(
        self,
        df: pl.DataFrame,
        items_df: pl.DataFrame,
        item_id_col: str = "item_id"
    ) -> None:
        self.items_embeddings = np.array(items_df["embedding"].to_list())
        self.items_item_ids = items_df[item_id_col].to_numpy()

        # find the most liked item for each user
        self.train_most_liked_item = (
            df.lazy()
            .sort("rating", descending=True)
            .group_by("user_id")
            .agg(pl.col(item_id_col).first().alias(item_id_col))
            .join(
                items_df.lazy().select([item_id_col, "embedding"]),
                on=item_id_col,
                how="left"
            )
            .collect()
        )
        self.trained = True

    def _get_batch_predictions(
        self,
        batch_user_embeddings: np.ndarray,
        batch_current_items: np.ndarray,
        batch_user_mask: np.ndarray,
        topn: int
    ) -> list[np.ndarray]:
        batch_predictions = []
        
        # compute similarity scores for all items in batch
        batch_scores = batch_user_embeddings @ self.items_embeddings.T

        for user_scores, current_item, has_embedding in zip(
            batch_scores, batch_current_items, batch_user_mask
        ):
            if not has_embedding:
                batch_predictions.append([])
                continue

            # get top (topn + 1) most similar items
            top_indices = np.argpartition(-user_scores, topn + 1)[:topn + 1]
            
            # filter out current item and take topn
            recommended_items = []
            for item_idx in top_indices:
                if self.items_item_ids[item_idx] != current_item:
                    recommended_items.append(self.items_item_ids[item_idx])
                if len(recommended_items) >= topn:
                    break

            batch_predictions.append(recommended_items)

        return batch_predictions

    def predict(
        self,
        df: pl.DataFrame,
        topn: int = 10,
        batch_size: int = 5_000
    ) -> list[np.ndarray]:
        assert self.trained
        
        user_data = (
            df.select(["user_id"])
            .join(self.train_most_liked_item, on="user_id", how="left")
        )

        user_embeddings = np.array(user_data["embedding"].to_list())
        current_items = user_data["item_id"].to_numpy()
        user_mask = user_data["embedding"].is_not_null().to_numpy()

        predictions = []
        n_users = len(user_data)

        for batch_start in tqdm(range(0, n_users, batch_size)):
            batch_end = min(batch_start + batch_size, n_users)
            
            batch_predictions = self._get_batch_predictions(
                user_embeddings[batch_start:batch_end],
                current_items[batch_start:batch_end],
                user_mask[batch_start:batch_end],
                topn
            )
            predictions.extend(batch_predictions)

        return predictions


item_knn = ItemkNN()
item_knn.fit(train, books)
test = test.with_columns(
    item_knn_recs=pl.Series(item_knn.predict(test))
)
print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="item_knn_recs")

100%|██████████| 38/38 [01:38<00:00,  2.60s/it]


shape: (5, 4)
┌────────────────────────┬───────────────────────┬────────────────────────┬────────────────────────┐
│ user_id                ┆ item_id               ┆ toppopular_recs        ┆ item_knn_recs          │
│ ---                    ┆ ---                   ┆ ---                    ┆ ---                    │
│ str                    ┆ list[i64]             ┆ array[i64, 10]         ┆ list[i64]              │
╞════════════════════════╪═══════════════════════╪════════════════════════╪════════════════════════╡
│ 00000377eea48021d30027 ┆ [13252]               ┆ [4058, 15514, … 13159] ┆ [17956, 12482, …       │
│ 30d56aca…              ┆                       ┆                        ┆ 31641]                 │
│ 00009ab2ed8cbfceda5a59 ┆ [2328]                ┆ [4058, 15514, … 13159] ┆ [16282, 986, … 21243]  │
│ da409663…              ┆                       ┆                        ┆                        │
│ 00009e46d18f223a82b22d ┆ [28636, 30197]        ┆ [4058, 15514, … 13159] ┆ [

{'ndcg@10': 0.008,
 'recall@10': 0.008,
 'map@10': 0.003,
 'serendipity@10': 0.008,
 'coverage': 0.764}

## 1: Двухэтапная модель с LightGBM

Двухэтапный пайплайн, сочетающий генерацию кандидатов и их последующее ранжирование - попытка объединить базовые идеи и сделать лучше

1. Генерация кандидатов

Для каждого юзера $u$ делаем первоначальный набор кандидатов $C_u$:
   - топ-N рекомендаций от модели коллаборативной фильтрации ItemKNN
   - топ-N популярных айтемов 

$$ C_u = \text{ItemKNN}(u, N) \cup \text{TopPopular}(N) $$

2. Ранжирование с помощью LightGBM

Обучение на позитивных примерах (реальные взаимодействия (user, item) из train) и негативных примерах (hard negative sampling -  в качестве негативных примеров берутся айтемы, которые ItemKNN порекомендовала пользователю, но с которыми он в действительности не взаимодействовал)

Для каждого примера (user, item) формируем вектор признаков, включающий признаки пользователя (количество взаимодействий, средний рейтинг)и признаки товара (количество взаимодействий, средний рейтинг, эмбеддинги из CountVectorizer)

Для каждого пользователя $u$ и каждого кандидата $i \in C_u$ LightGBM вычисляет скор $S(u, i)$

In [12]:
import lightgbm as lgb
import joblib
import os

In [ ]:
class LightGBMReranker:
    def __init__(self, item_knn_recommender, popular_recommender, n_candidates=100):
        self.trained = False
        self.model = lgb.LGBMClassifier(random_state=42, n_estimators=200, learning_rate=0.1, num_leaves=31)
        self.item_knn = item_knn_recommender
        self.popular = popular_recommender
        self.n_candidates = n_candidates
        
        self.user_features = None
        self.item_features = None
        self.feature_names = None

    def _prepare_features(self, df: pl.DataFrame, items_df: pl.DataFrame):
        user_features = df.group_by("user_id").agg(
            user_interactions=pl.len(),
            user_avg_rating=pl.col("rating").mean()
        )
        item_features = df.group_by("item_id").agg(
            item_interactions=pl.len(),
            item_avg_rating=pl.col("rating").mean()
        )
        item_embeddings = items_df.select(["item_id", "embedding"])
        embedding_cols = [f"emb_{i}" for i in range(len(item_embeddings['embedding'][0]))]
        item_embeddings_wide = item_embeddings.with_columns(
            [pl.col("embedding").list.get(i).alias(f"emb_{i}") for i in range(len(embedding_cols))]
        ).drop("embedding")

        self.user_features = user_features
        self.item_features = item_features.join(item_embeddings_wide, on="item_id", how="left")

    def fit(self, df: pl.DataFrame, items_df: pl.DataFrame, n_neg_samples_per_pos: int = 4, max_samples: int = 2_000_000):
        self._prepare_features(df, items_df)
        
        positives = df.select(["user_id", "item_id"]).with_columns(label=pl.lit(1, dtype=pl.Int8))
        train_users_df = df.select("user_id").unique()
        knn_candidates_list = self.item_knn.predict(train_users_df, topn=self.n_candidates)
        
        user_id_col = []
        item_id_col = []
        for user_id, candidates in zip(train_users_df["user_id"].to_list(), knn_candidates_list):
            if len(candidates) > 0:
                user_id_col.extend([user_id] * len(candidates))
                item_id_col.extend(candidates)
        
        candidates_df = pl.DataFrame({"user_id": user_id_col, "item_id": item_id_col})
        negatives = candidates_df.join(positives, on=["user_id", "item_id"], how="anti")
        negatives = negatives.with_columns(label=pl.lit(0, dtype=pl.Int8))
        
        n_pos = len(positives)
        n_neg = len(negatives)
        if n_neg > n_pos * n_neg_samples_per_pos:
            negatives = negatives.sample(n=n_pos * n_neg_samples_per_pos)

        train_data = pl.concat([positives, negatives]).sample(fraction=1.0, shuffle=True)
        
        if len(train_data) > max_samples:
            train_data = train_data.sample(n=max_samples, shuffle=True)

        train_df = train_data.join(self.user_features, on="user_id", how="left")
        train_df = train_df.join(self.item_features, on="item_id", how="left").fill_null(0)

        y = train_df["label"]
        X = train_df.drop(["user_id", "item_id", "label"])
        self.feature_names = X.columns
        self.model.fit(X.to_numpy(), y.to_numpy())
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10) -> list[np.ndarray]:
        test_users_df = df.select("user_id").unique()
        candidates_knn_list = self.item_knn.predict(test_users_df, topn=self.n_candidates)
        candidates_popular = self.popular.predict(test_users_df, topn=self.n_candidates)[0]

        all_preds = []
        
        for i, user_id in enumerate(tqdm(test_users_df["user_id"].to_list())):
            user_candidates = set(candidates_knn_list[i]) | set(candidates_popular)
            if not user_candidates:
                all_preds.append(np.array([]))
                continue

            candidates_df = pl.DataFrame({"user_id": [user_id] * len(user_candidates), "item_id": list(user_candidates)})
            
            predict_df = candidates_df.join(self.user_features, on="user_id", how="left")
            predict_df = predict_df.join(self.item_features, on="item_id", how="left").fill_null(0)
    
            X_predict = predict_df[self.feature_names]
            scores = self.model.predict_proba(X_predict.to_pandas())[:, 1]
            
            results_df = pl.DataFrame({"item_id": predict_df["item_id"], "score": scores})
            top_items= results_df.sort("score", descending=True).head(topn)["item_id"].to_numpy()
            all_preds.append(top_items)
            
        return all_preds

    def save_model(self, path="models/lgbm_reranker.joblib"):
        if not os.path.exists("models"):
            os.makedirs("models")
        model_data= {'model': self.model, 'user_features': self.user_features, 'item_features': self.item_features, 'feature_names': self.feature_names}
        joblib.dump(model_data, path)

lgbm_reranker = LightGBMReranker(item_knn_recommender=item_knn, popular_recommender=toppop)
lgbm_reranker.fit(train, books)
lgbm_reranker.save_model()

100%|██████████| 70/70 [02:42<00:00,  2.33s/it]


[LightGBM] [Info] Number of positive: 517987, number of negative: 1482013
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9178
[LightGBM] [Info] Number of data points in the train set: 2000000, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258993 -> initscore=-1.051206
[LightGBM] [Info] Start training from score -1.051206


In [14]:
test_users_df = test.select("user_id").unique()
lgbm_preds = lgbm_reranker.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "lgbm_recs": pl.Series(lgbm_preds)
})
test = test.join(preds_df, on="user_id", how="left")
print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="lgbm_recs")

100%|██████████| 185828/185828 [28:17<00:00, 109.45it/s]


shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ user_id        ┆ item_id        ┆ random_recs    ┆ toppopular_re ┆ item_knn_recs ┆ lgbm_recs     │
│ ---            ┆ ---            ┆ ---            ┆ cs            ┆ ---           ┆ ---           │
│ str            ┆ list[i64]      ┆ array[i64, 10] ┆ ---           ┆ list[i64]     ┆ array[i64,    │
│                ┆                ┆                ┆ array[i64,    ┆               ┆ 10]           │
│                ┆                ┆                ┆ 10]           ┆               ┆               │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 00000377eea480 ┆ [13252]        ┆ [6469, 30479,  ┆ [4058, 15514, ┆ [17956,       ┆ [960, 27314,  │
│ 21d3002730d56a ┆                ┆ … 23297]       ┆ … 13159]      ┆ 12482, …      ┆ … 5729]       │
│ ca…            ┆                ┆                ┆               ┆ 31641]  

{'ndcg@10': 0.029,
 'recall@10': 0.023,
 'map@10': 0.012,
 'serendipity@10': 0.031,
 'coverage@k': 0.024}

In [16]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["lgbm_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
960,Fangirl,"This special edition includes fan art, a ribbon bookmark, a Q&A with the author, and an excerpt from her new book Carry On. In Rainbow Rowell's Fangirl, Cath is a Simon Snow fan. Okay, the whole world is a Simon Snow fan, but for Cath, being a fan is her life-and she's really good at it. She and her twin sister, Wren, ensconced themselves in the Simon Snow series when they were just kids; it's what got them through their mother leaving. Reading. Rereading. Hanging out in Simon Snow forums, writing Simon Snow fan fiction, dressing up like the characters for every movie premiere. Cath's sister has mostly grown away from fandom, but Cath can't let go. She doesn't want to. Now that they're going to college, Wren has told Cath she doesn't want to be roommates. Cath is on her own, completely outside of her comfort zone. She's got a surly roommate with a charming, always-around boyfriend, a fiction-writing professor who thinks fan fiction is the end of the civilized world, a handsome classmate who only wants to talk about words . . . And she can't stop worrying about her dad, who's loving and fragile and has never really been alone. For Cath, the question is: Can she do this? Can she make it without Wren holding her hand? Is she ready to start living her own life? And does she even want to move on if it means leaving Simon Snow behind? A New York Times Book Review Notable Children's Book of 2013 A New York TimesBest Seller!",
5398,Matilda,"Matilda is a little girl who is far too good to be true. At age five-and-a-half she's knocking off double-digit multiplication problems and blitz-reading Dickens. Even more remarkably, her classmates love her even though she's a super-nerd and the teacher's pet. But everything is not perfect in Matilda's world. For starters she has two of the most idiotic, self-centred parents who ever lived. Then there's the large, busty nightmare of a school mistress, Mrs (""The"") Trunchbull, a former hammer- throwing champion who flings children at will and is approximately as sympathetic as a bulldozer. Fortunately for Matilda, she has the inner resources to deal with such annoyances: astonishing intelligence, saintly patience and an innate talent for revenge. She warms up with some practical jokes aimed at her hapless parents, but the true test comes when she rallies in defence of her teacher, the sweet Miss Honey, against the diabolical Trunchbull. There is never any doubt that Matilda will carry the day. Even so, this wonderful story is far from predictable--the big surprise comes when Matilda discovers a new, mysterious facet of her mental dexterity. Roald Dahl, while keeping the plot moving imaginatively, also has an unerring ear for emotional truth. The truly reader cares about Matilda because, in addition to all her other gifts, she has real feelings.",
5729,Heartless,"In this modern-day take on the classic Frankenstein tale, as told from the monster's perspective, Jolene Hall is dead--sort of. She can walk, think and talk, but her heart doesn't beat and her lungs stopped breathing ages ago. After Jo is abducted and subjected to horrific experiments, she wakes up to find her body is a mosaic of jagged wounds and stapled flesh. Jo Hall has a choice: turn herself in to the authorities, or team up with her roommate Lucy and her boyfriend Eli to find a way to save herself. To Jo, the choice is clear. She'd like to know who turned her into a monster, and she'd like to live to see another sunrise. But that choice has drastic repercussions. On a trip deep into the snowy White Mountains, to a hidden laboratory filled with danger and bodies of the dead, Jo and Lucy find more 'creatures' just like her. Part body, part machine, run by batteries and electricity, these girls are killers, created by a shadowy Order with a penchant for chaos...and murder. To make matters worse, a photo on a wall of victims reveals Lucy is next in line to be ""recruited"" into this army 

In [52]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["lgbm_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
960,Fangirl,"This special edition includes fan art, a ribbon bookmark, a Q&A with the author, and an excerpt from her new book Carry On. In Rainbow Rowell's Fangirl, Cath is a Simon Snow fan. Okay, the whole world is a Simon Snow fan, but for Cath, being a fan is her life-and she's really good at it. She and her twin sister, Wren, ensconced themselves in the Simon Snow series when they were just kids; it's what got them through their mother leaving. Reading. Rereading. Hanging out in Simon Snow forums, writing Simon Snow fan fiction, dressing up like the characters for every movie premiere. Cath's sister has mostly grown away from fandom, but Cath can't let go. She doesn't want to. Now that they're going to college, Wren has told Cath she doesn't want to be roommates. Cath is on her own, completely outside of her comfort zone. She's got a surly roommate with a charming, always-around boyfriend, a fiction-writing professor who thinks fan fiction is the end of the civilized world, a handsome classmate who only wants to talk about words . . . And she can't stop worrying about her dad, who's loving and fragile and has never really been alone. For Cath, the question is: Can she do this? Can she make it without Wren holding her hand? Is she ready to start living her own life? And does she even want to move on if it means leaving Simon Snow behind? A New York Times Book Review Notable Children's Book of 2013 A New York TimesBest Seller!",
5398,Matilda,"Matilda is a little girl who is far too good to be true. At age five-and-a-half she's knocking off double-digit multiplication problems and blitz-reading Dickens. Even more remarkably, her classmates love her even though she's a super-nerd and the teacher's pet. But everything is not perfect in Matilda's world. For starters she has two of the most idiotic, self-centred parents who ever lived. Then there's the large, busty nightmare of a school mistress, Mrs (""The"") Trunchbull, a former hammer- throwing champion who flings children at will and is approximately as sympathetic as a bulldozer. Fortunately for Matilda, she has the inner resources to deal with such annoyances: astonishing intelligence, saintly patience and an innate talent for revenge. She warms up with some practical jokes aimed at her hapless parents, but the true test comes when she rallies in defence of her teacher, the sweet Miss Honey, against the diabolical Trunchbull. There is never any doubt that Matilda will carry the day. Even so, this wonderful story is far from predictable--the big surprise comes when Matilda discovers a new, mysterious facet of her mental dexterity. Roald Dahl, while keeping the plot moving imaginatively, also has an unerring ear for emotional truth. The truly reader cares about Matilda because, in addition to all her other gifts, she has real feelings.",
5729,Heartless,"In this modern-day take on the classic Frankenstein tale, as told from the monster's perspective, Jolene Hall is dead--sort of. She can walk, think and talk, but her heart doesn't beat and her lungs stopped breathing ages ago. After Jo is abducted and subjected to horrific experiments, she wakes up to find her body is a mosaic of jagged wounds and stapled flesh. Jo Hall has a choice: turn herself in to the authorities, or team up with her roommate Lucy and her boyfriend Eli to find a way to save herself. To Jo, the choice is clear. She'd like to know who turned her into a monster, and she'd like to live to see another sunrise. But that choice has drastic repercussions. On a trip deep into the snowy White Mountains, to a hidden laboratory filled with danger and bodies of the dead, Jo and Lucy find more 'creatures' just like her. Part body, part machine, run by batteries and electricity, these girls are killers, created by a shadowy Order with a penchant for chaos...and murder. To make matters worse, a photo on a wall of victims reveals Lucy is next in line to be ""recruited"" into this army 

По метрикам - хуже TopPopular, но лучше остальных моделей (исключая coverage).Тут походу появились айтемы, рекомендуемые всем, которые не относятся к самым популярным - судя по serendipity, холодных айтемов стало больше, по сравнению с предыдущими моделями, но мы полностью пожертвовали точностью остальных рекомендаций. Короче, делаем дальше, перебор

## 2: Матричная факторизация: SVD

In [13]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

Коллаборативная фильтрация на основе SVD: используем SVD для матричной факторизации

1.  Исходная матрица взаимодействий $R$ (размера $|U| \times |I|$) аппроксимируется произведением двух матриц меньшей размерности: матрицы латентных векторов пользователей $P$ и айтемов $Q$
    $$R \approx P \times Q^T$$
2.  Скор релевантности для пары (user, item)  $\hat{r}_{ui} = \langle p_u,q_i \rangle $, далее сортируем по скору

In [13]:
class TruncatedSVDRecommender:
    def __init__(self, n_components=100, random_state=42):
        self.trained = False
        self.model = TruncatedSVD(n_components=n_components, random_state=random_state)
        
        self.user_map, self.item_map = None, None
        self.user_embeddings, self.item_embeddings = None, None
        self.train_interactions = None
        self.item_inv_map_array = None

    def fit(self, df: pl.DataFrame):
        interactions_df = df.group_by("user_id").agg(pl.col("item_id"))
    
        self.train_interactions = dict(zip(
            interactions_df["user_id"].to_list(), 
            interactions_df["item_id"].to_list()
        ))

        self.user_map = {uid: i for i, uid in enumerate(df["user_id"].unique())}
        self.item_map = {iid: i for i, iid in enumerate(df["item_id"].unique())}
        
        item_inv_map = {i: iid for iid, i in self.item_map.items()}
        self.item_inv_map_array = np.array([item_inv_map[i] for i in range(len(item_inv_map))])

        rows = df["user_id"].replace(self.user_map)
        cols = df["item_id"].replace(self.item_map)
        values = df["rating"] if "rating" in df.columns else np.ones(len(df))
        
        interactions_matrix = csr_matrix((values, (rows, cols)), shape=(len(self.user_map), len(self.item_map)))
        self.user_embeddings = self.model.fit_transform(interactions_matrix)
        self.item_embeddings = self.model.components_.T
        
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10, batch_size: int = 4096) -> list[np.ndarray]:
        test_user_ids = df["user_id"].unique().to_list()
        
        final_preds = [np.array([])] * len(test_user_ids)
        
        user_jobs = [(i, self.user_map.get(uid)) for i, uid in enumerate(test_user_ids)]
        known_user_jobs = [(orig_idx, user_idx) for orig_idx, user_idx in user_jobs if user_idx is not None]

        if not known_user_jobs:
            return final_preds
        
        original_indices, known_user_indices = zip(*known_user_jobs)

        for i in tqdm(range(0, len(known_user_indices), batch_size), desc="Predicting in batches"):
            batch_user_indices = known_user_indices[i : i + batch_size]
            batch_original_indices = original_indices[i : i + batch_size]
            
            batch_scores = self.user_embeddings[list(batch_user_indices)] @ self.item_embeddings.T

            batch_user_ids = [test_user_ids[orig_idx] for orig_idx in batch_original_indices]
            for j, user_id in enumerate(batch_user_ids):
                seen_items = self.train_interactions.get(user_id, [])
                if seen_items: 
                    seen_indices = [self.item_map[iid] for iid in seen_items if iid in self.item_map]
                    batch_scores[j, seen_indices] = -np.inf

            top_indices = np.argpartition(-batch_scores, topn, axis=1)[:, :topn]
            top_scores = batch_scores[np.arange(batch_scores.shape[0])[:, None], top_indices]
            sorted_within_top_indices = np.argsort(-top_scores, axis=1)
            final_batch_indices = top_indices[np.arange(top_indices.shape[0])[:, None], sorted_within_top_indices]
            
            recommendations = self.item_inv_map_array[final_batch_indices]
            
            for j, orig_idx in enumerate(batch_original_indices):
                final_preds[orig_idx] = recommendations[j]
                
        return final_preds

svd_rec = TruncatedSVDRecommender(n_components=128)
svd_rec.fit(train)

test_users_df = test.select("user_id").unique()
svd_preds = svd_rec.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "svd_recs": pl.Series(svd_preds)
})

test = test.join(preds_df, on="user_id", how="left")

print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="svd_recs")

Predicting in batches: 100%|██████████| 46/46 [04:50<00:00,  6.31s/it]


shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ user_id        ┆ item_id        ┆ random_recs    ┆ toppopular_re ┆ item_knn_recs ┆ svd_recs      │
│ ---            ┆ ---            ┆ ---            ┆ cs            ┆ ---           ┆ ---           │
│ str            ┆ list[i64]      ┆ array[i64, 10] ┆ ---           ┆ list[i64]     ┆ array[i64,    │
│                ┆                ┆                ┆ array[i64,    ┆               ┆ 10]           │
│                ┆                ┆                ┆ 10]           ┆               ┆               │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 00000377eea480 ┆ [13252]        ┆ [10746, 32113, ┆ [4058, 15514, ┆ [17956,       ┆ [22896,       │
│ 21d3002730d56a ┆                ┆ … 22743]       ┆ … 13159]      ┆ 12482, …      ┆ 22898, …      │
│ ca…            ┆                ┆                ┆               ┆ 31641]  

{'ndcg@10': 0.011,
 'recall@10': 0.007,
 'map@10': 0.004,
 'serendipity@10': 0.012,
 'coverage@k': 0.038}

In [14]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["svd_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
1471,"Crank (Crank, #1)","In Crank, Ellen Hopkins chronicles the turbulent and often disturbing relationship between Kristina, a character based on her own daughter, and the ""monster,"" the highly addictive drug crystal meth, or ""crank."" Kristina is introduced to the drug while visiting her largely absent and ne'er-do-well father. While under the influence of the monster, Kristina discovers her sexy alter-ego, Bree: ""there is no perfect daughter, / no gifted high school junior, / no Kristina Georgia Snow. / There is only Bree."" Bree will do all the things good girl Kristina won't, including attracting the attention of dangerous boys who can provide her with a steady flow of crank. Soon, her grades plummet, her relationships with family and friends deteriorate, and she needs more and more of the monster just to get through the day. Kristina hits her lowest point when she is raped by one of her drug dealers and becomes pregnant as a result. Her decision to keep the baby slows her drug use, but doesn't stop it, and the author leaves the reader with the distinct impression that Kristina/Bree may never be free from her addiction.",
2683,"The Sisterhood of the Traveling Pants (Sisterhood, #1)","FOUR VERY DIFFERENT FRIENDS. ONE PAIR OF MAGICAL PANTS. AND A SUMMER APART ... We, the Sisterhood, hereby instate the following rules to govern the use of the Traveling Pants: 1. You must never wash the Pants. 2. You must never double-cuff the Pants. It's tacky. There will never be a time when this will not be tacky. 3. You must never say the word ""phat"" while wearing the Pants. You must also never think ""I am fat"" while wearing the Pants. 4. You must never let a boy take off the Pants (although you may take them off yourself in his presence). 5. You must not pick your nose while wearing the Pants. You may, however, scratch casually at your nostril while really kind of picking. 6. Upon our reunion, you must follow the proper procedures for documenting your time in the Pants. 7. You must write to your Sisters throughout the summer, no matter how much fun you are having without them. 8. You must pass the Pants along to your Sister according to the specifications set down by the Sisterhood. Failures to comply will result in a sever spanking upon our reunion. 9. You must not wear the Pants with a tucked-in shirt and belt. See rule #2. 10. Remember: Pants = love. Love your pals. Love yourself.",
2870,"The Princess Diaries (The Princess Diaries, #1)","She's just a New York City girl living with her artist mom.... But, as it turns out, her Dad is the Prince of Genovia (So that's why a limo meets her at the airport). Shock of the Century! Like it or not, Mia Thermopolis is prime princess material. Mia must take princess lessons from her dreaded grandme re, the dowager princess of Genovia, who thinks Mia has a thing or two to learn before she steps up to the throne. Well, her father can lecture her until he's royal-blue in the face about her princessly duty -- no way is she moving to Genovia and leaving Manhattan behind. But what's a girl to do when her name is Princess Amelia Mignonette Grimaldi Thermopolis Renaldo? The Princess Diaries was among the 2001 Quick Picks for Reluctant Young Readers (American Library Association), Books for the Teen Age 2001 (New York Public Library), and the 2001 Best Books for Young Adults (American Library Association).""",
7097,This Is Where It Ends,"10:00 a.m.The principal of Opportunity High School finishes her speech, welcoming the entire student body to a new semester and encouraging them to excel and achieve. 10:02 a.m.The students get up to leave the auditorium for their next class. 10:03 a.m.The auditorium doors won't open. 10:05 a.m.Someone starts shooting. Told from four different perspectives over the span of fifty-four harrowing minutes, terror reigns as one student's calculated revenge turns into the ultimate game of survival.",
7375,Hate List,"The powerful and 

In [15]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["svd_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
2889,Simon vs. the Homo Sapiens Agenda,"Sixteen-year-old and not-so-openly gay Simon Spier prefers to save his drama for the school musical. But when an email falls into the wrong hands, his secret is at risk of being thrust into the spotlight. Now Simon is actually being blackmailed: if he doesn't play wingman for class clown Martin, his sexual identity will become everyone's business. Worse, the privacy of Blue, the pen name of the boy he's been emailing, will be compromised. With some messy dynamics emerging in his once tight-knit group of friends, and his email correspondence with Blue growing more flirtatious every day, Simon's junior year has suddenly gotten all kinds of complicated. Now, change-averse Simon has to find a way to step out of his comfort zone before he's pushed out--without alienating his friends, compromising himself, or fumbling a shot at happiness with the most confusing, adorable guy he's never met.",
3625,"Will Grayson, Will Grayson","This is an Alternate Cover Edition for ISBN-13: 978-0142418475 ; ISBN-10: 0142418471 Will Grayson meets Will Grayson. One cold night, in a most unlikely corner of Chicago, two strangers are about to cross paths. From that moment on, their world will collide and Iives intertwine. It's not that far from Evanston to Naperville, but Chiacago suburbanites Will Grayson and Will Grayson might as well live on different planets. When fate delivers them both to the same surprising crossroads, the Will Graysons find their lives overlapping and hurtling in new and unexpected directions. With a push from friends new and old - including the massive, and massively fabulous, Tiny Cooper, offensive lineman and musical theater auteur extraordinaire - Will and Will begin building toward respective romantic turns-of-heart and the epic production of history's most awesome high school musical.",
8303,"Every Day (Every Day, #1)","Every day a different body. Every day a different life. Every day in love with the same girl. There's never any warning about where it will be or who it will be. A has made peace with that, even established guidelines by which to live: Never get too attached. Avoid being noticed. Do not interfere. It's all fine until the morning that A wakes up in the body of Justin and meets Justin's girlfriend, Rhiannon. From that moment, the rules by which A has been living no longer apply. Because finally A has found someone he wants to be with--day in, day out, day after day.",
10019,The Disreputable History of Frankie Landau-Banks,"Frankie Landau-Banks, at age 16: No longer the kind of girl to take ""no"" for an answer and possibly a criminal mastermind. This is the story of how she got that way. Frankie Landau-Banks at age 14: Debate Club. Her father's ""bunny rabbit."" A mildly geeky girl attending a highly competitive boarding school. Frankie Landau-Banks at age 15: A knockout figure. A sharp tongue. A chip on her shoulder. And a gorgeous new senior boyfriend: the supremely goofy, word-obsessed Matthew Livingston. Frankie Landau-Banks. No longer the kind of girl to take ""no"" for an answer. Especially when ""no"" means she's excluded from her boyfriend's all-male secret society. Not when her ex-boyfriend shows up in the strangest of places. Not when she knows she's smarter than any of them. When she knows Matthew's lying to her. And when there are so many, many pranks to be done. Frankie Landau-Banks, at age 16: Possibly a criminal mastermind. This is the story of how she got that way.",
14981,Eleanor and Park,"Two misfits. One extraordinary love. Eleanor... Red hair, wrong clothes. Standing behind him until he turns his head. Lying beside him until he wakes up. Making everyone else seem drabber and flatter and never good enough...Eleanor. Park... He knows she'll love a song before he plays it for her. He laughs at her jokes before she ever gets to the punch line. There's a place on his chest, just below his throat, that makes her want to 

Метрики получились низкие, но и надеяться на нормальное качество рекомендаций тут наверное и не стоило, попробуем улучшить. Модель предлагает много книг в рамках одной и той же серии, но честно говоря, не вижу объяснения, почему coverage получается настолько низким, постараемся сделать и его лучше в том числе

In [17]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.8 MB/s  0:00:01 eta 0:00:01


Гибридная рекомендательная система с взвешенным скорингом

Модель вычисляет итоговую релевантность товара для пользователя, объединяя рекомендаций из трех различных источников - с целью попробовать поднять coverage 

1. Итоговый скор $S_{final}$ для юзера $u$ и айтема $i$ формируется как взвешенная сумма нормализованных оценок

$$S_{final}(u, i) = w_{collab} \cdot \text{norm}(S_{collab}) + w_{content} \cdot \text{norm}(S_{content}) + w_{pop} \cdot \text{norm}(S_{pop})$$

2. Составляющие
    * Коллаборативная фильтрация ($S_{collab}$):SVD, обученный на матрице взаимодействий user-item. Вычисляется как скалярное произведение латентных векторов \(p_u \cdot q_i\) (тут только теплые айтемы)

    * Контентная фильтрация ($S_{content}$): усредняем контентные эмбеддинги книг с рейтингом, для ускорения используется FAISS (здесь все айтемы)

    * Наиболее популярные: скор на основе логарифма числа взаимодействий

In [14]:
import faiss 
from sklearn.preprocessing import minmax_scale

class MultiChannelRecommender:
    def __init__(self, n_components=128, random_state=42, weights=None):
        if weights is None:
            self.weights = {'collab': 1.0, 'content': 0.7, 'pop': 0.2}
        else:
            self.weights = weights
            
        self.svd = TruncatedSVD(n_components=n_components, random_state=random_state)
        self.trained = False
        self.user_map, self.warm_item_map, self.all_item_map = None, None, None
        self.user_embeddings, self.warm_item_embeddings = None, None
        self.user_content_profiles, self.all_item_embeddings = None, None
        self.pop_scores = None
        self.train_interactions = None
        self.all_inv_map_array = None
        self.warm_item_indices_in_all = None
        self.faiss_index = None 

    def fit(self, df: pl.DataFrame, items_df: pl.DataFrame):
        interactions_df = df.group_by("user_id").agg(pl.col("item_id"))
        self.train_interactions = dict(zip(interactions_df["user_id"].to_list(), interactions_df["item_id"].to_list()))

        warm_items = df["item_id"].unique().to_list()
        all_items = items_df["item_id"].unique().to_list()
        
        self.user_map = {uid: i for i, uid in enumerate(df["user_id"].unique())}
        self.warm_item_map = {iid: i for i, iid in enumerate(warm_items)}
        self.all_item_map = {iid: i for i, iid in enumerate(all_items)}
        self.all_inv_map_array = np.array(all_items)
        self.warm_item_indices_in_all = np.array([self.all_item_map[iid] for iid in warm_items])

        rows = df["user_id"].replace(self.user_map)
        cols = df["item_id"].replace(self.warm_item_map)
        values = df["rating"] if "rating" in df.columns else np.ones(len(df))
        interactions_matrix = csr_matrix((values, (rows, cols)), shape=(len(self.user_map), len(self.warm_item_map)))
        self.user_embeddings = self.svd.fit_transform(interactions_matrix)
        self.warm_item_embeddings = self.svd.components_.T

        item_emb_map = {row["item_id"]: row["embedding"] for row in items_df.iter_rows(named=True)}
        self.all_item_embeddings = np.array(items_df["embedding"].to_list()).astype('float32')
        
        # нормализуем эмбеддинги для FAISS (L2-норма)
        faiss.normalize_L2(self.all_item_embeddings)
        
        #FAISS индекс
        d = self.all_item_embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(d) 
        self.faiss_index.add(self.all_item_embeddings)
        
        self.user_content_profiles = np.zeros((len(self.user_map), d), dtype='float32')
        for user_id, user_idx in self.user_map.items():
            user_items = self.train_interactions.get(user_id, [])
            if user_items:
                embs = [item_emb_map[iid] for iid in user_items if iid in item_emb_map]
                if embs: self.user_content_profiles[user_idx] = np.mean(embs, axis=0)
        
        # нормализуем профили пользователей
        faiss.normalize_L2(self.user_content_profiles)
        
        pop_counts_df = df['item_id'].value_counts()
        pop_counts = dict(zip(pop_counts_df["item_id"].to_list(), pop_counts_df["count"].to_list()))
        self.pop_scores = np.log1p([pop_counts.get(iid, 0) for iid in all_items]).astype('float32')

        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10, batch_size: int = 2048) -> list[np.ndarray]:
        test_user_ids = df["user_id"].unique().to_list()
        final_preds = [np.array([])] * len(test_user_ids)
        
        user_jobs = [(i, self.user_map.get(uid)) for i, uid in enumerate(test_user_ids)]
        known_user_jobs = [(orig_idx, user_idx) for orig_idx, user_idx in user_jobs if user_idx is not None]

        if not known_user_jobs: return final_preds
        
        original_indices, known_user_indices = zip(*known_user_jobs)
        for i in tqdm(range(0, len(known_user_indices), batch_size), desc="Predicting in batches"):
            batch_user_indices = list(known_user_indices[i : i + batch_size])
            batch_original_indices = list(original_indices[i : i + batch_size])
            batch_size_current = len(batch_user_indices)
            
            collab_scores = self.user_embeddings[batch_user_indices] @ self.warm_item_embeddings.T
            
            #кандидат и их оценки от FAISS
            profiles = self.user_content_profiles[batch_user_indices]
            content_scores_raw, content_indices = self.faiss_index.search(profiles, 50)
            minmax_scale(collab_scores, axis=1, copy=False)
            pop_scores_norm = minmax_scale(self.pop_scores)
            
            final_scores = np.zeros((batch_size_current, len(self.all_item_map)))
            rows = np.arange(batch_size_current)[:, np.newaxis]
            final_scores[rows, content_indices] = self.weights['content'] * content_scores_raw

            #добавляем остальные оценки
            final_scores += self.weights['pop'] * pop_scores_norm
            final_scores[:, self.warm_item_indices_in_all] += self.weights['collab'] * collab_scores

            batch_user_ids = [test_user_ids[orig_idx] for orig_idx in batch_original_indices]
            for j, user_id in enumerate(batch_user_ids):
                seen_items = self.train_interactions.get(user_id, [])
                if seen_items:
                    seen_indices = [self.all_item_map[iid] for iid in seen_items if iid in self.all_item_map]
                    final_scores[j, seen_indices] = -np.inf

            top_indices = np.argpartition(-final_scores, topn, axis=1)[:, :topn]
            recs = self.all_inv_map_array[top_indices]
            
            top_scores = final_scores[np.arange(batch_size_current)[:, None], top_indices]
            sorted_indices = np.argsort(-top_scores, axis=1)
            final_recs = recs[np.arange(batch_size_current)[:, None], sorted_indices]

            for j, orig_idx in enumerate(batch_original_indices):
                final_preds[orig_idx] = final_recs[j]
                
        return final_preds

mc_rec = MultiChannelRecommender(n_components=128)
mc_rec.fit(train, books)

test_users_df = test.select("user_id").unique()
mc_preds = mc_rec.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "mc_recs": pl.Series(mc_preds)
})

test = test.join(preds_df, on="user_id", how="left")

print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="mc_recs")

Predicting in batches: 100%|██████████| 91/91 [06:18<00:00,  4.16s/it]


shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ user_id        ┆ item_id        ┆ random_recs    ┆ toppopular_re ┆ item_knn_recs ┆ mc_recs       │
│ ---            ┆ ---            ┆ ---            ┆ cs            ┆ ---           ┆ ---           │
│ str            ┆ list[i64]      ┆ array[i64, 10] ┆ ---           ┆ list[i64]     ┆ array[i64,    │
│                ┆                ┆                ┆ array[i64,    ┆               ┆ 10]           │
│                ┆                ┆                ┆ 10]           ┆               ┆               │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 00000377eea480 ┆ [13252]        ┆ [21638, 33394, ┆ [4058, 15514, ┆ [17956,       ┆ [17311,       │
│ 21d3002730d56a ┆                ┆ … 13087]       ┆ … 13159]      ┆ 12482, …      ┆ 18150, …      │
│ ca…            ┆                ┆                ┆               ┆ 31641]  

{'ndcg@10': 0.014,
 'recall@10': 0.009,
 'map@10': 0.006,
 'serendipity@10': 0.013,
 'coverage': 0.254}

In [15]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["mc_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
8092,Tiger Lily,"Before Peter Pan belonged to Wendy, he belonged to the girl with the crow feather in her hair. . . . Fifteen-year-old Tiger Lily doesn't believe in love stories or happy endings. Then she meets the alluring teenage Peter Pan in the forbidden woods of Neverland and immediately falls under his spell. Peter is unlike anyone she's ever known. Impetuous and brave, he both scares and enthralls her. As the leader of the Lost Boys, the most fearsome of Neverland's inhabitants, Peter is an unthinkable match for Tiger Lily. Soon, she is risking everything--her family, her future--to be with him. When she is faced with marriage to a terrible man in her own tribe, she must choose between the life she's always known and running away to an uncertain future with Peter. With enemies threatening to tear them apart, the lovers seem doomed. But it's the arrival of Wendy Darling, an English girl who's everything Tiger Lily is not, that leads Tiger Lily to discover that the most dangerous enemies can live inside even the most loyal and loving heart. From the New York Times bestselling author of Peaches comes a magical and bewitching story of the romance between a fearless heroine and the boy who wouldn't grow up.",
8303,"Every Day (Every Day, #1)","Every day a different body. Every day a different life. Every day in love with the same girl. There's never any warning about where it will be or who it will be. A has made peace with that, even established guidelines by which to live: Never get too attached. Avoid being noticed. Do not interfere. It's all fine until the morning that A wakes up in the body of Justin and meets Justin's girlfriend, Rhiannon. From that moment, the rules by which A has been living no longer apply. Because finally A has found someone he wants to be with--day in, day out, day after day.",
13726,"Library of Souls (Miss Peregrine’s Peculiar Children, #3)","A New York Times #1 best seller A boy with extraordinary powers. An army of deadly monsters. An epic battle for the future of peculiardom. The adventure that began with Miss Peregrine's Home for Peculiar Childrenand continued in Hollow Citycomes to a thrilling conclusion with Library of Souls. As the story opens, sixteen-year-old Jacob discovers a powerful new ability, and soon he's diving through history to rescue his peculiar companions from a heavily guarded fortress. Accompanying Jacob on his journey are Emma Bloom, a girl with fire at her fingertips, and Addison MacHenry, a dog with a nose for sniffing out lost children. They'll travel from modern-day London to the labyrinthine alleys of Devil's Acre, the most wretched slum in all of Victorian England. It's a place where the fate of peculiar children everywhere will be decided once and for all. Like its predecessors, Library of Soulsblends thrilling fantasy with never-before-published vintage photography to create a one-of-a-kind reading experience. From the Hardcover edition.",
16498,"The Wrath and the Dawn (The Wrath and the Dawn, #1)","This is an alternate cover edition for . One Life to One Dawn. In a land ruled by a murderous boy-king, each dawn brings heartache to a new family. Khalid, the eighteen-year-old Caliph of Khorasan, is a monster. Each night he takes a new bride only to have a silk cord wrapped around her throat come morning. When sixteen-year-old Shahrzad's dearest friend falls victim to Khalid, Shahrzad vows vengeance and volunteers to be his next bride. Shahrzad is determined not only to stay alive, but to end the caliph's reign of terror once and for all. Night after night, Shahrzad beguiles Khalid, weaving stories that enchant, ensuring her survival, though she knows each dawn could be her last. But something she never expected begins to happen: Khalid is nothing like what she'd imagined him to be. This monster is a boy with a tormented heart. Incredibly, Shahrzad finds herself falling in love. How is this possible? It's an unforgivable betrayal. 

In [17]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["mc_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
1166,"Snow Like Ashes (Snow Like Ashes, #1)","A heartbroken girl. A fierce warrior. A hero in the making. Sixteen years ago the Kingdom of Winter was conquered and its citizens enslaved, leaving them without magic or a monarch. Now, the Winterians' only hope for freedom is the eight survivors who managed to escape, and who have been waiting for the opportunity to steal back Winter's magic and rebuild the kingdom ever since. Orphaned as an infant during Winter's defeat, Meira has lived her whole life as a refugee, raised by the Winterians' general, Sir. Training to be a warrior--and desperately in love with her best friend, and future king, Mather -- she would do anything to help her kingdom rise to power again. So when scouts discover the location of the ancient locket that can restore Winter's magic, Meira decides to go after it herself. Finally, she's scaling towers, fighting enemy soldiers, and serving her kingdom just as she's always dreamed she would. But the mission doesn't go as planned, and Meira soon finds herself thrust into a world of evil magic and dangerous politics - and ultimately comes to realize that her destiny is not, never has been, her own. Sara Raasch's debut fantasy is a lightning-fast tale of loyalty, love, and finding one's destiny.",
5237,"The Revenge of Seven (Lorien Legacies, #5)","The fifth book of the #1 New York Timesbestselling I Am Number Four series! The Garde have suffered an unbearable loss. Number Five has betrayed them. Eight is gone forever. Ella has been kidnapped. The others are now scattered. In Chicago, John makes the unlikeliest of allies: a Mogadorian named Adam, who has switched sides. Adam knows the Mogs secrets, and he knows where to hit them: their command base near Washington, DC. But during the assault, John learns he and the Garde might be too late. The Mogadorians have commenced their ultimate invasion plans. Meanwhile, Six, Nine, and Marina make their way through the Florida Everglades, hot on the trail of the traitorous Five. With the development of a new Legacy, Marina finally has the power to fight back--if her thirst for revenge doesn't consume her first. The Garde may be broken and divided once more, but they will not be defeated. As long as one still stands, the battle for Earth's survival is not lost. Michael Bay, director of Transformers, raved: ""Number Four is a hero for this generation."" This epic young adult series is perfect for fans of action-packed science fiction like The Fifth Wave series by Rick Yancey, The Maze Runner series by James Dashner, and Orson Scott Card's Ender's Game. The battle for Earth's survival wages on. Read the rest of the series: #6: The Fate of Ten, and #7: United as One. Don't miss the first book in the brand-new I Am Number Four spin-off series: Generation One.",
13738,"Half Bad (The Half Bad Trilogy, #1)","approx 8h 36m Wanted by no one. Hunted by everyone. Sixteen-year-old Nathan lives in a cage: beaten, shackled, trained to kill. In a modern-day England where two warring factions of witches live amongst humans, Nathan is an abomination, the illegitimate son of the world's most terrifying and violent witch, Marcus. Nathan's only hope for survival is to escape his captors, track down Marcus, and receive the three gifts that will bring him into his own magical powers--before it's too late. But how can Nathan find his father when there is no one safe to trust, not even family, not even the girl he loves? Half Badis an international sensation and the start of a brilliant trilogy: a gripping tale of alienation and the indomitable will to survive.",
13774,"Storm Siren (Storm Siren, #1)","""I raise my chin as the buyers stare. Yes. Look. You don't want me. Because, eventually, accidentally, I will destroy you."" In a world at war, a slave girl's lethal curse could become one kingdom's weapon of salvation. If the curse--and the girl--can be controlled. As a slave in the war-weary kingdom of Faelen, seve

Пока, мне кажется, наиболее удачное сочетание метрик получилось - coverage стал сильно более адекватным, по recall, precision, ndcg тоже улучшили обычный SVD, но в меньшей степени. Глазами видно, что рекомендации разнообразные и вроде бы адекватные, хотя и разрозненные и с уклоном в мистику/детективы/триллеры. Попробую чуть подкрутить веса

In [18]:
mc_rec = MultiChannelRecommender(n_components=128, weights={'collab': 0.8, 'content': 0.8, 'pop': 0.3})
mc_rec.fit(train, books)

test_users_df = test.select("user_id").unique()
mc_preds = mc_rec.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "mc_recs_2": pl.Series(mc_preds)
})

test = test.join(preds_df, on="user_id", how="left")

print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="mc_recs_2")

Predicting in batches: 100%|██████████| 91/91 [10:41<00:00,  7.05s/it]


shape: (5, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ user_id      ┆ item_id     ┆ random_recs ┆ toppopular_ ┆ item_knn_re ┆ mc_recs     ┆ mc_recs_2   │
│ ---          ┆ ---         ┆ ---         ┆ recs        ┆ cs          ┆ ---         ┆ ---         │
│ str          ┆ list[i64]   ┆ array[i64,  ┆ ---         ┆ ---         ┆ array[i64,  ┆ array[i64,  │
│              ┆             ┆ 10]         ┆ array[i64,  ┆ list[i64]   ┆ 10]         ┆ 10]         │
│              ┆             ┆             ┆ 10]         ┆             ┆             ┆             │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 00000377eea4 ┆ [13252]     ┆ [21638,     ┆ [4058,      ┆ [17956,     ┆ [17311,     ┆ [1164,      │
│ 8021d3002730 ┆             ┆ 33394, …    ┆ 15514, …    ┆ 12482, …    ┆ 18150, …    ┆ 21429, …    │
│ d56aca…      ┆             ┆ 13087]      ┆ 13159]      ┆ 31641]      ┆ 1831

{'ndcg@10': 0.015,
 'recall@10': 0.009,
 'map@10': 0.007,
 'serendipity@10': 0.014,
 'coverage': 0.246}

В целом, примерно то же самое

In [24]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["mc_recs_2"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
2804,"The Unbecoming of Mara Dyer (Mara Dyer, #1)","Mara Dyer doesn't think life can get any stranger. She wakes from a coma in hospital with no memory of how she got there or of the bizarre accident that caused the deaths of her best friends and her boyfriend, yet left her mysteriously unharmed. The doctors suggest that starting over in a new city, a new school, would be good for her and just to let the memories gradually come back on their own. But Mara's new start is anything but comforting. She sees the faces of her dead friends everywhere, and when she suddenly begins to see other people's deaths right before they happen, Mara wonders whether she's going crazy! And if dealing with all this wasn't enough, Noah Shaw, the most beautiful boy she has ever seen can't seem to leave her alone... but as her life unravels around her, Mara can't help but wonder if Noah has another agenda altogether...",
9454,"Ruin and Rising (The Grisha, #3)","The capital has fallen. The Darkling rules Ravka from his shadow throne. Now the nation's fate rests with a broken Sun Summoner, a disgraced tracker, and the shattered remnants of a once-great magical army. Deep in an ancient network of tunnels and caverns, a weakened Alina must submit to the dubious protection of the Apparat and the zealots who worship her as a Saint. Yet her plans lie elsewhere, with the hunt for the elusive firebird and the hope that an outlaw prince still survives. Alina will have to forge new alliances and put aside old rivalries as she and Mal race to find the last of Morozova's amplifiers. But as she begins to unravel the Darkling's secrets, she reveals a past that will forever alter her understanding of the bond they share and the power she wields. The firebird is the one thing that stands between Ravka and destruction--and claiming it could cost Alina the very future she's fighting for. Ruin and Risingis the thrilling final installment in Leigh Bardugo's Grisha Trilogy.",
16096,"Siege and Storm (The Grisha, #2)","The second book in The Grisha Trilogy by the #1 New York Timesbestselling author of Six of Crows. Darkness never dies. Hunted across the True Sea, haunted by the lives she took on the Fold, Alina must try to make a life with Mal in an unfamiliar land, all while keeping her identity as the Sun Summoner a secret. But she can't outrun her past or her destiny for long. The Darkling has emerged from the Shadow Fold with a terrifying new power and a dangerous plan that will test the very boundaries of the natural world. With the help of a notorious privateer, Alina returns to the country she abandoned, determined to fight the forces gathering against Ravka. But as her power grows, Alina slips deeper into the Darkling's game of forbidden magic, and farther away from Mal. Somehow, she will have to choose between her country, her power, and the love she always thought would guide her--or risk losing everything to the oncoming storm. Siege and Storm is the second book in The Grisha Trilogy by Leigh Bardugo. Book one, Shadow and Bone, is a New York Timesbestseller, and book 3, Ruin and Rising, is an Amazon Best Book of the Year and a USA Todaybestseller. This title has Common Core connections, and this deluxe paperback edition features bonus materials like an interview with Leigh Bardugo, a bonus short story, ""The Tailor,"" and more.",
17256,"Shadow and Bone (The Grisha, #1)","Alina Starkov doesn't expect much from life. Orphaned by the Border Wars, the one thing she could rely on was her best friend and fellow refugee, Mal. And lately not even that seems certain. Drafted into the army of their war-torn homeland, they're sent on a dangerous mission into the Fold, a swath of unnatural darkness crawling with monsters who feast on human flesh. When their convoy is attacked, all seems lost until Alina reveals a dormant power that not even she knew existed. Ripped from everything she knows, she is whisked away to the royal court to be tra

In [25]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["mc_recs_2"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
3395,"Twilight / Life and Death (Twilight, #1, 1.75)","WHEN ISABELLA SWAN MOVES TO THEgloomy town of Forks and meets the mysterious, alluring Edward Cullen, her life takes a thrilling and terrifying turn. With his porcelain skin, golden eyes, mesmerizing voice, and supernatural gifts, Edward is both irresistible and enigmatic. WHAT BELLA DOESN'T REALIZE IS THE CLOSERshe gets to him, the more she is putting herself and those around her at risk. And, it might be too late to turn back... DEEPLY SEDUCTIVE AND EXTRAORDINARILYsuspenseful, Twilighthas enraptured millions and become a modern classic, redefining genres within young adult literature and inspiring a phenomenon that has had readers yearning for more. This special tenth anniversary dual edition includes a foreword by the author as well as a complete reimagining of the original novel. Turn this book over to read Life and Death. / WHEN BEAUFORT SWAN MOVES TO THEgloomy town of Forks and meets the mysterious, alluring Edythe Cullen, his life takes a thrilling and terrifying turn. With her porcelain skin, golden eyes, mesmerizing voice, and supernatural gifts, Edythe is both irresistible and enigmatic. WHAT BEAU DOESN'T REALIZE IS THEcloser he gets to her, the more he is putting himself and those around him at risk. And, it might be too late to turn back... IN CELEBRATION OF THE TENTH ANNIVERSARYof Twilight, Stephenie Meyer has crafted Life and Death, a bold and compelling reimagining of the iconic love story that will surprise and enthrall readers. This special dual edition includes a foreword by the author as well as the complete original novel. Turn this book over to read Twilight.",
9238,Sanctum (Asylum #2),"In this chilling, fast-paced sequel to the New York Timesbestselling Asylum, which Publishers Weeklycalled ""a strong YA debut that reveals the enduring impact of buried trauma on a place,"" three teens must return to the asylum that still haunts their dreams to end the nightmare once and for all. With the page-turning suspense and horror that made Asylumsuch a standout, and featuring found photographs from real vintage carnivals, Sanctumis a mind-bending reading experience that's perfect for fans of the smash hit Miss Peregrine's Home for Peculiar Children. Dan, Abby, and Jordan remain traumatized by the summer they shared in the Brookline asylum. Much as they'd love to move on, many questions remain, and someone is determined to keep the terror alive, sending the teens photos of an old-timey carnival, with no note and no name. Forsaking their plan never to go back, the teens return to New Hampshire College under the guise of a weekend for prospective students, and there they realize that the carnival from the photos is not only real, it's here on campus, apparently for the first time in many years. Sneaking away from sample classes and college parties, Dan and his friends lead a tour of their own--one through the abandoned houses and hidden places of a surrounding town. Camford is hiding a terrible past, and the truth behind Dan's connection to the asylum's evil warden is more terrifying than Dan ever imagined.",
9454,"Ruin and Rising (The Grisha, #3)","The capital has fallen. The Darkling rules Ravka from his shadow throne. Now the nation's fate rests with a broken Sun Summoner, a disgraced tracker, and the shattered remnants of a once-great magical army. Deep in an ancient network of tunnels and caverns, a weakened Alina must submit to the dubious protection of the Apparat and the zealots who worship her as a Saint. Yet her plans lie elsewhere, with the hunt for the elusive firebird and the hope that an outlaw prince still survives. Alina will have to forge new alliances and put aside old rivalries as she and Mal race to find the last of Morozova's amplifiers. But as she begins to unravel the Darkling's secrets, she reveals a past that will forever alter her understanding of the bond they share and the power she wields. The firebi

## 3: Линейные модели: EASE (SANSA)

In [28]:
!pip install numpy==2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.2 MB/s  0:00:02 eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


In [10]:
#EASE (Embarrassingly Shallow Autoencoder) через SANSA
from scipy.sparse import csr_matrix
from sansa import SANSA, SANSAConfig, ICFGramianFactorizerConfig, UMRUnitLowerTriangleInverterConfig

class EASERecommender:
    def __init__(self, l2=500.0, weight_matrix_density=7e-5):
        factorizer_config = ICFGramianFactorizerConfig()
        inverter_config = UMRUnitLowerTriangleInverterConfig()
        config = SANSAConfig(
            l2=l2,
            weight_matrix_density=weight_matrix_density,
            gramian_factorizer_config=factorizer_config,
            lower_triangle_inverter_config=inverter_config
        )
        self.model = SANSA(config)
        self.trained = False
        
        self.user_map = None
        self.item_map = None
        self.train_interactions = None
        self.item_inv_map_array = None
        self.popular_items_indices = None

    def fit(self, df: pl.DataFrame):
        interactions_df = df.group_by("user_id").agg(pl.col("item_id"))
        self.train_interactions = dict(zip(
            interactions_df["user_id"].to_list(),
            interactions_df["item_id"].to_list()
        ))
        
        self.user_map = {uid: i for i, uid in enumerate(df["user_id"].unique())}
        self.item_map = {iid: i for i, iid in enumerate(df["item_id"].unique())}
        
        item_inv_map = {i: iid for iid, i in self.item_map.items()}
        self.item_inv_map_array = np.array([item_inv_map[i] for i in range(len(item_inv_map))])

        popular_items = df['item_id'].value_counts().head(20)['item_id'].to_list()
        self.popular_items_indices = [self.item_map[iid] for iid in popular_items if iid in self.item_map]

        rows = df["user_id"].replace(self.user_map)
        cols = df["item_id"].replace(self.item_map)
        values = np.ones(len(df))
        
        interactions_matrix = csr_matrix((values, (rows, cols)), shape=(len(self.user_map), len(self.item_map)))
        self.model.fit(interactions_matrix)
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10) -> list[np.ndarray]:
        assert self.trained
        
        test_user_ids = df["user_id"].unique().to_list()
        
        test_user_map = {uid: i for i, uid in enumerate(test_user_ids)}
        
        rows, cols, data = [], [], []
        for user_id, row_idx in test_user_map.items():
            user_history = self.train_interactions.get(user_id, [])
            
            #дополняем короткие истории популярными 
            if 0 < len(user_history) < topn:
                history_indices = {self.item_map[iid] for iid in user_history if iid in self.item_map}
                
                #добавляем популярные, пока история не станет достаточно длинной
                for pop_idx in self.popular_items_indices:
                    if len(history_indices) >= topn: break
                    history_indices.add(pop_idx)
                
                for item_idx in history_indices:
                    rows.append(row_idx)
                    cols.append(item_idx)
                    data.append(1)
            
            #для пользователей с достаточной историей или без оставляем как есть
            elif len(user_history) >= topn:
                 for item_id in user_history:
                    if item_id in self.item_map:
                        rows.append(row_idx)
                        cols.append(self.item_map[item_id])
                        data.append(1)

        test_interactions_matrix = csr_matrix(
            (data, (rows, cols)),
            shape=(len(test_user_ids), len(self.item_map))
        )
        
        top_k_indices, _ = self.model.recommend(test_interactions_matrix, k=topn, mask_input=True)
        recommendations = self.item_inv_map_array[top_k_indices]
        return list(recommendations)

In [11]:
ease_rec = EASERecommender()
ease_rec.fit(train)
test_users_df = test.select("user_id").unique()
ease_preds = ease_rec.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "ease_recs": pl.Series(ease_preds)
})

test = test.join(preds_df, on="user_id", how="left")

INFO:sansa.model:Computing column norms of X^TX...
INFO:sansa.model:Scaling columns of X by computed norms...
INFO:sansa.model:Computing LDL^T decomposition of permuted item-item matrix...
INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 500.0*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (31300, 31300) 
                    nnz = 264972836 
                    density = 27.046600% 
                    size = 2119.9 MB
                
                Attempting incomplete factorization of a relatively dense matrix (27.046600% dense). 
                This is unstable:
                 - the factorization might fail and automatically restart with additional regularization
                 - the resulting approximate factor might be of lesser quality
                You may want to tr

In [12]:
print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="ease_recs")

shape: (5, 5)
┌───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────┐
│ user_id           ┆ item_id           ┆ random_recs       ┆ toppopular_recs   ┆ ease_recs        │
│ ---               ┆ ---               ┆ ---               ┆ ---               ┆ ---              │
│ str               ┆ list[i64]         ┆ array[i64, 10]    ┆ array[i64, 10]    ┆ array[i64, 10]   │
╞═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪══════════════════╡
│ 00000377eea48021d ┆ [13252]           ┆ [4549, 3786, …    ┆ [4058, 15514, …   ┆ [27003, 15514, … │
│ 3002730d56aca…    ┆                   ┆ 23]               ┆ 13159]            ┆ 24671]           │
│ 00009ab2ed8cbfced ┆ [2328]            ┆ [17711, 20522, …  ┆ [4058, 15514, …   ┆ [11475, 10108, … │
│ a5a59da409663…    ┆                   ┆ 14156]            ┆ 13159]            ┆ 15193]           │
│ 00009e46d18f223a8 ┆ [28636, 30197]    ┆ [30088, 5896, …   ┆ [4058, 15514, …

{'ndcg@10': 0.021,
 'recall@10': 0.016,
 'map@10': 0.009,
 'serendipity@10': 0.018,
 'coverage': 0.398}

In [13]:
ease_rec = EASERecommender(l2=250, weight_matrix_density=2e-4)
ease_rec.fit(train)

INFO:sansa.model:Computing column norms of X^TX...
INFO:sansa.model:Scaling columns of X by computed norms...
INFO:sansa.model:Computing LDL^T decomposition of permuted item-item matrix...
INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 250*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (31300, 31300) 
                    nnz = 264972836 
                    density = 27.046600% 
                    size = 2119.9 MB
                
                Attempting incomplete factorization of a relatively dense matrix (27.046600% dense). 
                This is unstable:
                 - the factorization might fail and automatically restart with additional regularization
                 - the resulting approximate factor might be of lesser quality
                You may want to try 

In [14]:
test_users_df = test.select("user_id").unique()
ease_preds = ease_rec.predict(test_users_df)

preds_df = pl.DataFrame({
    "user_id": test_users_df["user_id"],
    "ease_recs": pl.Series(ease_preds)
})

test = test.join(preds_df, on="user_id", how="left")

print(test.head())
evaluate_recommender_modified(df=test, model_preds_col="ease_recs_right")

shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ user_id        ┆ item_id        ┆ random_recs    ┆ toppopular_re ┆ ease_recs     ┆ ease_recs_rig │
│ ---            ┆ ---            ┆ ---            ┆ cs            ┆ ---           ┆ ht            │
│ str            ┆ list[i64]      ┆ array[i64, 10] ┆ ---           ┆ array[i64,    ┆ ---           │
│                ┆                ┆                ┆ array[i64,    ┆ 10]           ┆ array[i64,    │
│                ┆                ┆                ┆ 10]           ┆               ┆ 10]           │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 00000377eea480 ┆ [13252]        ┆ [4549, 3786, … ┆ [4058, 15514, ┆ [27003,       ┆ [33370,       │
│ 21d3002730d56a ┆                ┆ 23]            ┆ … 13159]      ┆ 15514, …      ┆ 28386, …      │
│ ca…            ┆                ┆                ┆               ┆ 24671]  

{'ndcg@10': 0.028,
 'recall@10': 0.023,
 'map@10': 0.013,
 'serendipity@10': 0.021,
 'coverage': 0.264}

P.S: столбец ease_recs содержит результаты запуска модели с параметрами по умолчанию, ease_recs_right - с другими (l2=250, density=2e-4), попытка сделать модель чуть сложнее. Как видно, получили прирост всех метрик, кроме coverage - получается, хотя и модель находит больше сложных взаимосвязей, рекомендует она все-таки меньшую часть каталога. Заметно, что есть трейд офф между чрезмерной рекомендацией холодных айтемов и теплых

Сравним результаты 

ease_recs - наиболее разреженная и быстрая модель

In [15]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["ease_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
3792,"Thomas’s First Memory of the Flare (Maze Runner, #2.5)","Short flashback that occurs between ""The Scorch Trials"" and ""The Death Cure.""",
7478,The Red Pyramid (Kane Chronicles #1),"From the author of the blockbuster best-selling series Percy Jackson and the Olympianscomes the first installment of the Kane Chronicles, combining nonstop action, humor, and suspense in an epic tale of loyalty and heroism. Since his mother's death six years ago, Carter Kane has been living out of a suitcase, traveling the globe with his father, the brilliant Egyptologist Dr. Julius Kane. But while Carter's been homeschooled, his younger sister, Sadie, has been living with their grandparents in London. Sadie has just what Carter wants -- school friends and a chance at a ""normal"" life. But Carter has just what Sadie longs for -- time with their father. After six years of living apart, the siblings have almost nothing in common. Until now. On Christmas Eve, Sadie and Carter are reunited when their father brings them to the British Museum, with a promise that he's going to ""make things right."" But all does not go according to plan; Carter and Sadie watch as Julius summons a mysterious figure, who quickly banishes their father and causes a fiery explosion. Soon Carter and Sadie discover that the gods of Ancient Egypt are waking, and the worst of them -- Set -- has a frightening scheme. To save their father, they must embark on a dangerous journey -- a quest that brings them ever closer to the truth about their family and its links to the House of Life, a secret order that has existed since the time of the pharaohs. An unabridged recording on 12 CDs (14 hrs, 42 mins).",
15513,"The Scorch Trials (The Maze Runner, #2)",Solving the Maze was supposed to be the end. Thomas was sure that escape from the Maze would mean freedom for him and the Gladers. But WICKED isn't done yet. Phase Two has just begun. The Scorch. There are no rules. There is no help. You either make it or you die. The Gladers have two weeks to cross through the Scorch--the most burned-out section of the world. And WICKED has made sure to adjust the variables and stack the odds against them. Friendships will be tested. Loyalties will be broken. All bets are off. There are others now. Their survival depends on the Gladers' destruction--and they're determined to survive.,
15728,"United as One (Lorien Legacies, #7)","The seventh and final book in the #1 New York Times bestselling I Am Number Four series! With United as One, this action-packed series comes to a surprising, breathtaking, and utterly satisfying conclusion. The Garde didn't start this war, but they'll do whatever it takes to end it once and for all. . . . The Mogadorians have invaded Earth. Their warships loom over our most populated cities, and no country will risk taking them head on. The Garde are all that stand in the way, but they'll need an army of their own to win this fight. They've teamed up with the US military, but it might not be enough. The Garde need reinforcements, and they've found them in the most unexpected place. Teenagers from across the globe, like John Smith's best friend, Sam, have developed abilities. So John and the others must get to them before the Mogs, because if they don't their enemies will use these gifted teens for their own sinister plan. But after all the Mogadorians have taken from John--his home, his family, his friends, and the person he loves most--he might not want to put any more lives in danger. He's got nothing left to lose, and he's just discovered he has been given an incredible new Legacy. Now he can turn himself into the ultimate weapon. So will he risk his life to save the world, or will he realize that power in numbers will save us all?",
18406,The Kill Order,"Read the fourth book in the #1 New York Timesbestselling Maze Runner series, perfect for fans of The Hunger Gamesand Divergent. The first book, The Maze Runner, is now a major mot

In [16]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["ease_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
4911,"Sweet, Hereafter (Heaven, #3)",Coretta Scott King Award-winner Angela Johnson concludes her Heaven trilogy with a poignant tale of discovering where--and with whom--you belong.,
5222,"Predator's Gold (The Hungry City Chronicles, #2)","The Jenny was drifting across the shoulder of a big volcano. Beyond it there were no more mountains, just an endless blue-white plain stretching to the horizon. They were at the mercy of the wind, and it was carrying them helplessly into the Ice Wastes. After two years of carefree traveling in the Jenny Haniver, Tom and Hester find themselves back in danger. Fleeing from the grim aviators of the Green Storm, they stumble onto the ice city of Anchorage just in time. But Anchorage is not a safe refuge: Devastated by plague, and haunted by thieves -- or perhaps ghosts -- the city is barely lurching along. The savage Huntsmen of Arkangel are closing in, and the young margravine must make a last desperate bid for survival. She sets a course for the Dead Continent -- America ... In this breathtaking sequel to the award-winning Mortal Engines, Philip Reeve plunges us into a ruthless and terrifyingly believable world where cities eat each other, betrayal is as common as the blasted land the cities traverse, and loyalty holds the only chance of survival.",
8245,"The Opal Deception (Artemis Fowl, #4)","The evil pixie Opal Koboi has spent the last year in a self-induced coma, plotting her revenge on all those who foiled her attempt to destroy the LEPrecon fairy police. And Artemis Fowl is at the top of her list. After his last run-in with the fairies, Artemis had his mind wiped of his memories of the world belowground. But they have not forgotten about him. Once again, he must stop the human and fairy worlds from colliding--only this time, Artemis faces an enemy who may have finally outsmarted him.",
10389,Running Out of Time,"Run For Your Life Jessie lives with her family in the frontier village of Clifton, Indiana, in 1840 -- or so she believes. When diphtheria strikes the village and the children of Clifton start dying, Jessie's mother reveals a shocking secret -- it's actually 1996, and they are living in a reconstructed village that serves as a tourist site. In the world outside, medicine exists that can cure the dread disease, and Jessie's mother is sending her on a dangerous mission to bring back help. But beyond the walls of Clifton, Jessie discovers a world even more alien and threatening than she could have imagined, and soon she finds her own life in jeopardy. Can she get help before the children of Clifton, and Jessie herself, run out of time?",
15514,"The Hunger Games (The Hunger Games, #1)","Winning will make you famous. Losing means certain death. The nation of Panem, formed from a post-apocalyptic North America, is a country that consists of a wealthy Capitol region surrounded by 12 poorer districts. Early in its history, a rebellion led by a 13th district against the Capitol resulted in its destruction and the creation of an annual televised event known as the Hunger Games. In punishment, and as a reminder of the power and grace of the Capitol, each district must yield one boy and one girl between the ages of 12 and 18 through a lottery system to participate in the games. The 'tributes' are chosen during the annual Reaping and are forced to fight to the death, leaving only one survivor to claim victory. When 16-year-old Katniss's young sister, Prim, is selected as District 12's female representative, Katniss volunteers to take her place. She and her male counterpart Peeta, are pitted against bigger, stronger representatives, some of whom have trained for this their whole lives. , she sees it as a death sentence. But Katniss has been close to death before. For her, survival is second nature.",
19710,"The First Part Last (Heaven, #2)","Bobby's a classic urban teenager. He's restless. He's impulsive. But the thing that makes him different is this: He's

ease_recs_right - менее разреженная и быстрая модель

In [18]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["ease_recs_right"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
1164,My Life Next Door,"""One thing my mother never knew, and would disapprove of most of all, was that I watched the Garretts. All the time."" The Garretts are everything the Reeds are not. Loud, messy, affectionate. And every day from her rooftop perch, Samantha Reed wishes she was one of them . . . until one summer evening, Jase Garrett climbs up next to her and changes everything. As the two fall fiercely for each other, stumbling through the awkwardness and awesomeness of first love, Jase's family embraces Samantha - even as she keeps him a secret from her own. Then something unthinkable happens, and the bottom drops out of Samantha's world. She's suddenly faced with an impossible decision. Which perfect family will save her? Or is it time she saved herself? A transporting debut about family, friendship, first romance, and how to be true to one person you love without betraying another.",
12211,"The Transfer (Divergent, #0.1)","Complete your Divergent library with the Four stories! Fans of the Divergent series by #1 New York Times bestselling author Veronica Roth will be captivated by ""Four: The Transfer,"" the first of four stories, each between fifty and seventy-five pages long, set in the world of Divergent and told from Tobias's perspective. In ""The Transfer,"" readers witness Tobias's aptitude test, Choosing Day, and the moment he is given the infamous nickname ""Four.""",
13064,"Throne of Glass (Throne of Glass, #1)","There is an alternate cover edition for this ASIN . After serving out a year of hard labor in the salt mines of Endovier for her crimes, 18-year-old assassin Celaena Sardothien is dragged before the Crown Prince. Prince Dorian offers her her freedom on one condition: she must act as his champion in a competition to find a new royal assassin. Her opponents are men-thieves and assassins and warriors from across the empire, each sponsored by a member of the king's council. If she beats her opponents in a series of eliminations, she'll serve the kingdom for three years and then be granted her freedom. Celaena finds her training sessions with the captain of the guard, Westfall, challenging and exhilirating. But she's bored stiff by court life. Things get a little more interesting when the prince starts to show interest in her... but it's the gruff Captain Westfall who seems to understand her best. Then one of the other contestants turns up dead... quickly followed by another. Can Celaena figure out who the killer is before she becomes a victim? As the young assassin investigates, her search leads her to discover a greater destiny than she could possibly have imagined.",
19293,"The Blood of Olympus (The Heroes of Olympus, #5)","The Blood of Olympus is the fifth book in the bestselling Heroes of Olympusseries - set in the high-octane world of Percy Jackson. Though the Greek and Roman crew members of the Argo II have made progress in their many quests, they still seem no closer to defeating the earth mother, Gaea. Her giants have risen - all of them - and they're stronger than ever. They must be stopped before the Feast of Spes, when Gaea plans to have two demigods sacrificed in Athens. She needs their blood -the blood of Olympus - in order to wake. The demigods are having more frequent visions of a terrible battle at Camp Half-Blood. The Roman legion from Camp Jupiter, led by Octavian, is almost within striking distance. Though it is tempting to take the Athena Parthenos to Athens to use as a secret weapon, the friends know that the huge statue belongs back on Long Island, where it might be able to stop a war between the two camps. The Athena Parthenos will go west; the Argo II will go east. The gods, still suffering from multiple personality disorder, are useless. How can a handful of young demigods hope to persevere against Gaea's army of powerful giants? As dangerous as it is to head to Athens, they have no other option. They have sacrificed too much already. And if G

In [19]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["ease_recs_right"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
10260,"Saving Raphael Santiago (The Bane Chronicles, #6)","One of the more interesting jobs Magnus has ever been involved in... A distraught mother in 1950s New York hires Magnus to find her son, Raphael, but when Magnus finds him young Raphael is already beyond saving. Or is he? This standalone e-only short story illuminates the life of the enigmatic Magnus Bane, whose alluring personality populates the pages of the number-one New York Times bestselling series The Mortal Instruments and The Infernal Devices.",
13442,"The Rise of the Hotel Dumort (The Bane Chronicles, #5)","Magnus Bane watches the once-glamorous Hotel Dumont become something else altogether in 1970s New York City. One of ten adventures in The Bane Chronicles. Fifty years after the Jazz Age rise of the Hotel Dumont, immortal warlock Magnus Bane knows the Manhattan landmark is on the decline. The once-beautiful Hotel Dumont has fallen into a decayed thing, a ruin, as dead as a place can be. But the vampires don't mind... This standalone e-only short story illuminates the life of the enigmatic Magnus Bane, whose alluring personality populates the pages of the #1 New York Times bestselling series, The Mortal Instruments and The Infernal Devices series. This story in The Bane Chronicles, The Fall of the Hotel Dumort, is written by Maureen Johnson and Cassandra Clare.",
16918,"The Hammer of Thor (Magnus Chase and the Gods of Asgard, #2)","Thor's hammer is missing again. The thunder god has a disturbing habit of misplacing his weapon - the mightiest force in the Nine Worlds. But this time the hammer isn't just lost, it has fallen into enemy hands. If Magnus Chase and his friends can't retrieve the hammer quickly, the mortal worlds will be defenseless against an onslaught of giants. Ragnarok will begin. The Nine Worlds will burn. Unfortunately, the only person who can broker a deal for the hammer's return is the gods' worst enemy, Loki - and the price he wants is veryhigh.",
17377,"The Fall of the Hotel Dumort (The Bane Chronicles, #7)","Fifty years after the Jazz Age and the rise of the Hotel Dumort, immortal warlock Magnus Bane knows the Manhattan landmark is on the decline. The once-beautiful Hotel Dumort has fallen into a decayed thing, a ruin, as dead as a place can be. But the vampires don't mind... One of ten eBook short stories in The Bane Chronicles, a series about the enigmatic Magnus Bane from Cassandra Clare's internationally bestselling series The Infernal Devices and The Mortal Instruments.",
19080,"The Mark of Athena (The Heroes of Olympus, #3)","Annabeth is terrified. Just when she's about to be reunited with Percy--after six months of being apart, thanks to Hera--it looks like Camp Jupiter is preparing for war. As Annabeth and her friends Jason, Piper, and Leo fly in on the Argo II, she can't blame the Roman demigods for thinking the ship is a Greek weapon. With its steaming bronze dragon masthead, Leo's fantastical creation doesn't appear friendly. Annabeth hopes that the sight of their praetor Jason on deck will reassure the Romans that the visitors from Camp Half-Blood are coming in peace. And that's only one of her worries. In her pocket Annabeth carries a gift from her mother that came with an unnerving demand: Follow the Mark of Athena. Avenge me. Annabeth already feels weighed down by the prophecy that will send seven demigods on a quest to find--and close-- the Doors of Death. What more does Athena want from her? Annabeth's biggest fear, though, is that Percy might have changed. What if he's now attached to Roman ways? Does he still need his old friends? As the daughter of the goddess of war and wisdom, Annabeth knows she was born to be a leader, but never again does she want to be without Seaweed Brain by her side. Narrated by four different demigods, The Mark of Athena is an unforgettable journey across land and sea to Rome, where important discoveries, surprising sacrifices, and unspeakable horrors await.

Видно, что менее разреженная имеет в целом менее разнообразные рекомендации,но судя по метрикам чаще попадает в редкие рекомендации, но в целом обе из них получились неплохими

## 4: Item2item + BM25

Item-to-Item коллаборативная фильтрация с весами BM25

Item-to-Item подход, который генерирует рекомендации на основе сходства между книгами - используются веса BM25 (Best Matching 25), которые позволяют более точно оценить значимость взаимодействия пользователя с книгой, снижая влияние популярных книг

- Вычисление веса взаимодействия с помощью BM25: вес $W(u, i)$ для  пользователя $u$ и книги $i$ рассчитывается

    $$W(u, i) = \text{IDF}(i) \cdot \frac{\text{TF}(u, i) \cdot (k_1 + 1)}{\text{TF}(u, i) + k_1 \cdot \left(1 - b + b \cdot \frac{|D_u|}{\text{avgDL}}\right)}$$

    где
    - $|D_u|$ — количество книг, прочитанных пользователем $u$.
    -   $\text{avgDL}$ — среднее количество прочитанных книг по всем пользователям
    -   $k_1$ и $b$ — гиперпараметры, контролирующие влияние TF и длины документа

-  Поиск похожих книг: на основе матрицы взвешенных взаимодействий для каждой книги находится $N$ ближайших соседей. Сходство между книгами $i$ и $j$ рассчитывается как косинусное расстояние 

- Рекомендации генерируются так: для юзера $u$ берется вся история его взаимодействий $\{i_1, i_2, \dots, i_k\}$, для каждой книги находятся соседи и скор считается по формуле

    $$S(u, j) = \sum_{i \in \text{History}(u)} \text{sim}(i, j)$$

In [20]:
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

class BM25ItemKNNRecommender:
    def __init__(self, n_neighbors: int = 200, k1: float = 1.2, b: float = 0.75):
        self.n_neighbors = n_neighbors
        self.k1 = k1
        self.b = b
        self.trained = False
        
        self.user_to_idx: dict[t.Any, int] = {}
        self.item_to_idx: dict[t.Any, int] = {}
        self.idx_to_item: dict[int, t.Any] = {}
        self.user_item_matrix: csr_matrix | None = None
        self.popular_items: list[t.Any] = []
        
        self.item_neighbors: np.ndarray | None = None
        self.item_neighbor_sims: np.ndarray | None = None

    def _calculate_bm25_weights(self, df: pl.DataFrame, user_col: str, item_col: str) -> csr_matrix:
        # (document length) - количество интеракций на пользователя
        user_interactions = df.group_by(user_col).agg(pl.count().alias("dl"))
        # (document frequency) - количество пользователей на айтем
        item_interactions = df.group_by(item_col).agg(pl.count().alias("df"))
        # idf 
        n_users = df[user_col].n_unique()
        
        item_interactions = item_interactions.with_columns(
            idf = (1 + (n_users - pl.col("df") + 0.5) / (pl.col("df") + 0.5)).log()
        )
        
        avg_dl = user_interactions["dl"].mean()  
        interactions_with_stats = df.join(user_interactions, on=user_col, how="left")
        interactions_with_stats = interactions_with_stats.join(item_interactions, on=item_col, how="left")
        
        tf = 1.0 
        interactions_with_weights = interactions_with_stats.with_columns(
            bm25_weight = pl.col("idf") * (tf * (self.k1 + 1)) / 
                         (tf + self.k1 * (1 - self.b + self.b * pl.col("dl") / avg_dl))
        )
        rows = [self.user_to_idx.get(u) for u in interactions_with_weights[user_col]]
        cols = [self.item_to_idx.get(i) for i in interactions_with_weights[item_col]]
        valid_indices = [(r, c, w) for r, c, w in zip(rows, cols, interactions_with_weights["bm25_weight"]) if r is not None and c is not None]
        rows, cols, weights = zip(*valid_indices)
        return csr_matrix(
            (weights, (rows, cols)),
            shape=(len(self.user_to_idx), len(self.item_to_idx))
        )

    def fit(self, df: pl.DataFrame, items_df: pl.DataFrame, user_id_col: str = "user_id", item_id_col: str = "item_id") -> None:
        users = df[user_id_col].unique().to_list()
        items = items_df[item_id_col].unique().to_list()
        
        self.user_to_idx = {u: i for i, u in enumerate(users)}
        self.item_to_idx = {it: j for j, it in enumerate(items)}
        self.idx_to_item = {j: it for it, j in self.item_to_idx.items()}

        rows = [self.user_to_idx[u] for u in df[user_id_col] if u in self.user_to_idx]
        cols = [self.item_to_idx[i] for i in df[item_id_col] if i in self.item_to_idx]
        self.user_item_matrix = csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(len(users), len(items)))

        self.popular_items = (
            df.group_by(item_id_col).agg(pl.count())
            .sort("count", descending=True)
            .head(100)[item_id_col].to_list()
        )


        weighted_user_item_matrix = self._calculate_bm25_weights(df, user_id_col, item_id_col)
        item_matrix_bm25 = normalize(weighted_user_item_matrix.T, norm="l2", axis=1)
        knn_model = NearestNeighbors(
            n_neighbors=min(self.n_neighbors, item_matrix_bm25.shape[0]),
            metric="cosine", algorithm="brute"
        )
        knn_model.fit(item_matrix_bm25)
        dists, neigh = knn_model.kneighbors(item_matrix_bm25, n_neighbors=self.n_neighbors)
        self.item_neighbors = neigh
        self.item_neighbor_sims = 1.0 - dists
        self.trained = True

    def predict(self, df: pl.DataFrame, topn: int = 10) -> list[np.ndarray]:
        assert self.trained, "Model must be trained before calling predict."
        
        predictions = []
        for user_id in tqdm(df["user_id"].to_list(), desc="Predicting"):
            recs = []
            
            if user_id in self.user_to_idx:
                user_idx = self.user_to_idx[user_id]
                user_items_indices = self.user_item_matrix[user_idx].indices
                
                if len(user_items_indices) > 0:
                    candidate_scores = np.zeros(len(self.item_to_idx))
                    
                    for item_idx in user_items_indices:
                        neighbors = self.item_neighbors[item_idx]
                        similarities = self.item_neighbor_sims[item_idx]
                        candidate_scores[neighbors] += similarities
                    
                    candidate_scores[user_items_indices] = -1
                    top_indices = np.argpartition(-candidate_scores, topn)[:topn]
                    top_indices = top_indices[np.argsort(-candidate_scores[top_indices])]
                    recs = [self.idx_to_item[idx] for idx in top_indices]
            
            if len(recs) < topn:
                unseen_popular = [item for item in self.popular_items if item not in recs]
                recs.extend(unseen_popular)

            predictions.append(np.array(recs[:topn], dtype=np.int64))
        
        return predictions

In [21]:
bm25_knn = BM25ItemKNNRecommender(n_neighbors=200)
bm25_knn.fit(train, books)

test = test.with_columns(
    bm25_recs=pl.Series(bm25_knn.predict(test, topn=10))
)

/var/folders/7_/sz_ftshd1b1b9sppxlm9vj9m0000gn/T/ipykernel_19142/377436379.py:64: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  df.group_by(item_id_col).agg(pl.count())
/var/folders/7_/sz_ftshd1b1b9sppxlm9vj9m0000gn/T/ipykernel_19142/377436379.py:23: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  user_interactions = df.group_by(user_col).agg(pl.count().alias("dl"))
/var/folders/7_/sz_ftshd1b1b9sppxlm9vj9m0000gn/T/ipykernel_19142/377436379.py:25: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  item_interactions = df.group_by(item_col).agg(pl.count().alias("df"))
Predicting: 100%|██████████| 185828/185828 [00:43<00:00, 4255.94it/s]


In [30]:
print(test.head())

evaluate_recommender_modified(df=test, model_preds_col="bm25_recs")

shape: (5, 4)
┌────────────────────────┬───────────────────────┬────────────────────────┬────────────────────────┐
│ user_id                ┆ item_id               ┆ toppopular_recs        ┆ bm25_recs              │
│ ---                    ┆ ---                   ┆ ---                    ┆ ---                    │
│ str                    ┆ list[i64]             ┆ array[i64, 10]         ┆ array[i64, 10]         │
╞════════════════════════╪═══════════════════════╪════════════════════════╪════════════════════════╡
│ 00000377eea48021d30027 ┆ [13252]               ┆ [4058, 15514, … 13159] ┆ [12599, 118, … 27745]  │
│ 30d56aca…              ┆                       ┆                        ┆                        │
│ 00009ab2ed8cbfceda5a59 ┆ [2328]                ┆ [4058, 15514, … 13159] ┆ [20150, 32516, …       │
│ da409663…              ┆                       ┆                        ┆ 17509]                 │
│ 00009e46d18f223a82b22d ┆ [28636, 30197]        ┆ [4058, 15514, … 13159] ┆ [

{'ndcg@10': 0.09,
 'recall@10': 0.072,
 'map@10': 0.049,
 'serendipity@10': 0.081,
 'coverage': 0.615}

In [29]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["bm25_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
1274,"City of Lost Souls (The Mortal Instruments, #5)","What price is too high to pay, even for love? Plunge into fifth installment in the internationally bestselling Mortal Instruments series and ""prepare to be hooked"" (Entertainment Weekly)--now with a gorgeous new cover, a map, a new foreword, and exclusive bonus content! City of Lost Souls is a Shadowhunters novel. When Jace and Clary meet again, Clary is horrified to discover that the demon Lilith's magic has bound her beloved Jace together with her evil brother Sebastian, and that Jace has become a servant of evil. The Clave is out to destroy Sebastian, but there is no way to harm one boy without destroying the other. As Alec, Magnus, Simon, and Isabelle wheedle and bargain with Seelies, demons, and the merciless Iron Sisters to try to save Jace, Clary plays a dangerous game of her own. The price of losing is not just her own life, but Jace's soul. She's willing to do anything for Jace, but can she still trust him? Or is he truly lost? Love. Blood. Betrayal. Revenge. Darkness threatens to claim the Shadowhunters in the harrowing fifth book of the Mortal Instruments series.",
8039,"Fairest (The Lunar Chronicles, #3.5)","In this stunning bridge book between Cressand Winterin the bestselling Lunar Chronicles, Queen Levana's story is finally told. Mirror, mirror on the wall, Who is the fairest of them all? Fans of the Lunar Chronicles know Queen Levana as a ruler who uses her ""glamour"" to gain power. But long before she crossed paths with Cinder, Scarlet, and Cress, Levana lived a very different story - a story that has never been told . . . until now. Marissa Meyer spins yet another unforgettable tale about love and war, deceit and death. This extraordinary book includes full-color art and an excerpt from Winter, the next book in the Lunar Chronicles series.",
9722,City of Ashes (The Mortal Instruments #2),"Is love worth betraying everything? Plunge into the second adventure in the internationally bestselling Mortal Instruments series and ""prepare to be hooked"" (Entertainment Weekly). Clary Fray just wishes that her life would go back to normal. But what's normal when you're a demon-slaying Shadowhunter, your mother is in a magically induced coma, and you can suddenly see Downworlders like werewolves, vampires, and faeries? If Clary left the world of the Shadowhunters behind, it would mean more time with her best friend, Simon, who's becoming more than a friend. But the Shadowhunting world isn't ready to let her go--especially her handsome, infuriating, newfound brother, Jace. And Clary's only chance to help her mother is to track down rogue Shadowhunter Valentine, who is probably insane, certainly evil--and also her father. To complicate matters, someone in New York City is murdering Downworlder children. Is Valentine behind the killings--and if he is, what is he trying to do? When the second of the Mortal Instruments, the Soul-Sword, is stolen, the terrifying Inquisitor arrives to investigate and zooms right in on Jace. How can Clary stop Valentine if Jace is willing to betray everything he believes in to help their father? In this breathtaking sequel to City of Bones, Cassandra Clare lures her readers back into the dark grip of New York City's Downworld, where love is never safe, and power becomes the deadliest temptation.",
9767,"Clockwork Princess (The Infernal Devices, #3)","Danger and betrayal, love and loss, secrets and enchantment are woven together in the breathtaking finale to the #1 New York Times bestselling Infernal Devices Trilogy, prequel to the internationally bestselling Mortal Instruments series. THE INFERNAL DEVICES WILL NEVER STOP COMING A net of shadows begins to tighten around the Shadowhunters of the London Institute. Mortmain plans to use his Infernal Devices, an army of pitiless automatons, to destroy the Shadowhunters. He needs only one last item to complete his plan: he needs Tessa Gray. Charlotte Branwel

In [14]:
HTML(dataframe_to_html(books.filter(pl.col("item_id").is_in(test.sample(1)["bm25_recs"][0].to_list())), ["item_id", "title", "description", "image_url"]))

item_id,title,description,image_url
3423,The Boy in the Black Suit,"A 2016 Coretta Scott King Author Honor book. Just when seventeen-year-old Matt thinks he can't handle one more piece of terrible news, he meets a girl who's dealt with a lot more--and who just might be able to clue him in on how to rise up when life keeps knocking him down--in this ""vivid, satisfying, and ultimately upbeat tale of grief, redemption, and grace"" (Kirkus Reviews) from the Coretta Scott King - John Steptoe Award-winning author of When I Was the Greatest. Matt wears a black suit every day. No, not because his mom died--although she did, and it sucks. But he wears the suit for his gig at the local funeral home, which pays way better than the Cluck Bucket, and he needs the income since his dad can't handle the bills (or anything, really) on his own. So while Dad's snagging bottles of whiskey, Matt's snagging fifteen bucks an hour. Not bad. But everything else? Not good. Then Matt meets Lovey. Crazy name, and she's been through more crazy stuff than he can imagine. Yet Lovey never cries. She's tough. Really tough. Tough in the way Matt wishes he could be. Which is maybe why he's drawn to her, and definitely why he can't seem to shake her. Because there's nothing more hopeful than finding a person who understands your loneliness--and who can maybe even help take it away.",
7209,American Street,"American Street is an evocative and powerful coming-of-age story perfect for fans of Everything, Everything; Bone Gap; and All American Boys. In this stunning debut novel, Pushcart-nominated author Ibi Zoboi draws on her own experience as a young Haitian immigrant, infusing this lyrical exploration of America with magical realism and vodouculture. On the corner of American Street and Joy Road, Fabiola Toussaint thought she would finally find une belle vie--a good life. But after they leave Port-au-Prince, Haiti, Fabiola's mother is detained by U.S. immigration, leaving Fabiola to navigate her loud American cousins, Chantal, Donna, and Princess; the grittiness of Detroit's west side; a new school; and a surprising romance, all on her own. Just as she finds her footing in this strange new world, a dangerous proposition presents itself, and Fabiola soon realizes that freedom comes at a cost. Trapped at the crossroads of an impossible choice, will she pay the price for the American dream?",
10547,Goodbye Stranger,"Masterly. . . . Sensitively explores togetherness, aloneness, betrayal and love. The New York Times Book Review 2016 Honor Book in Fiction, Boston Globe Horn Book Award A NYT Editors' Choice and NYT Notable Children's Books of 2015 This brilliant novel by Newbery Medal winner Rebecca Stead explores multiple perspectives on the bonds and limits of friendship. Bridge is an accident survivor who s wondering why she s still alive. Emily has new curves and an almost-boyfriend who wants a certain kind of picture. Tabitha sees through everybody s games or so she tells the world. The three girls are best friends with one rule: No fighting.Can it get them through seventh grade? This year everything is different for Sherm Russo as he gets to know Bridge Barsamian. What does it mean to fall for a girl as a friend? On Valentine s Day, an unnamed high school girl struggles with a betrayal. How long can she hide in plain sight? Each memorable character navigates the challenges of love and change in this captivating novel. Praise for Goodbye Stranger Six Starred Reviews Publishers WeeklyBest Books of 2015 School Library Journal Best Books of 2015 BooklistEditor s Choice 2015 The Horn Books Fanfare: Best Books of 2015 The Washington PostBest Books of 2015 The New York TimesNotable Children s Books of 2015 This astonishingly profound novel is not your average middle-school friendship tale. The Horn Book, Starred Stead shows how strongly love of all kinds can smooth the juddering path toward adulthood. Winsome, bighearted, and altogether rewarding. Booklist, Starred [Stea

Судя и по метрикам, и по самим рекомендациям, модель действительно получилась хорошая - высокий coverage сочетается с наилучшими полученными значениями и по остальным метрикам среди всех моделей. С применением веса, учитывающего популярность книги, получили желаемый результат - не испортив (и даже улучшив) рекомендации в целом, также рекомендуем и холодные айтемы, как и айтемы с небольшими взаимодействиями

## 5: Идея работы с эмбеддингами

Поработаем с эмбеддингами, потому что они тоже могут сильно влиять на качесто: рассмотрим каждое поле описания книги отдельно, для названий и описаний используем tf-idf, затем объединяем в матрицу и делаем LSA (снижаем размерность до 128)

In [31]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

books_for_embedding = books.with_columns(
    authors_str = pl.col("authors").list.eval(
        pl.lit("author_") + pl.element().struct.field(books.schema['authors'].inner.fields[0].name)
    ).list.join(" ").fill_null(""),
    
    tags_str = pl.col("tags").list.eval(
        pl.lit("tag_") + pl.element().str.replace_all("-", "_", literal=True).str.replace_all(" ", "_", literal=True)
    ).list.join(" ").fill_null("")
)

title_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))
title_vecs = title_vectorizer.fit_transform(books_for_embedding['title'].fill_null(""))

desc_vectorizer = TfidfVectorizer(max_features=15000, stop_words='english')
desc_vecs = desc_vectorizer.fit_transform(books_for_embedding['description'].fill_null(""))

categorical_text = books_for_embedding.select(
    pl.concat_str(["authors_str", "tags_str"], separator=" ")
).to_series()
cat_vectorizer = CountVectorizer(max_features=10000, token_pattern=r'[\w_]+')
cat_vecs = cat_vectorizer.fit_transform(categorical_text)

combined_sparse_matrix = hstack([title_vecs, desc_vecs, cat_vecs]).tocsr()

n_embedding_dims = 128
lsa = TruncatedSVD(n_components=n_embedding_dims, random_state=42)
dense_embeddings_raw = lsa.fit_transform(combined_sparse_matrix)

normalized_embeddings = normalize(dense_embeddings_raw, norm='l2', axis=1)

books = books.with_columns(
    pl.Series("embedding_new", normalized_embeddings.tolist())
)

display(books.head())

item_id,series,tags,title,description,url,image_url,authors,embedding_new
i64,list[str],list[str],str,str,str,str,list[struct[2]],list[f64]
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[0.550546, 0.033266, … -0.019082]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[0.585932, 0.176821, … -0.004595]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[0.5157, 0.278442, … -0.002641]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[0.638572, 0.326413, … -0.04838]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[0.324899, 0.076244, … -0.013045]"


## Что еще можно сделать, чтобы улучшить рекомендации?

1. Улучшить признаки, больше использовав метаданные книг, например

    - Семантические признаки из текста: использовать языковые модели (например, на основе BERT) для получения эмбеддингов для описаний и названий книг

    - Графовые признаки: построить граф, где узлами являются пользователи, книги, авторы и серии. Использовать, например, Node2Vec, чтобы получить графовые эмбеддинги - получится найти более сложные зависимости, возможно

    - Временные признаки: учесть динамику интересов пользователя и старение популярности книг, то есть как-то еще использовать date_added

2. Архитектура моделей

    - Нейтросетевые подходы 
    - Графовые нейронные сети (GNN) - использовать граф взаимодействий, возможно, позволило бы получить очень высокие результаты

3. Гибридные модели/ансамбли: даже исходя из имеющихся результатов, хорошо видно, что при объединении различных моделей их сильные стороны могут дать хороший результат

    - Переранжирование (Reranking): сгенерировать кандидатов, используя эффективную модель (например, BM25ItemKNNRecommender) для генерации широкого списка релевантных кандидатов. А затем обучить несколько моделей, и возможно, ансамблировать 